In [3]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [4]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ExpectedCond
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [5]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #-------------------------------------------------------------- ok
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(3)

    #--------------------------------------------------------------- ok
        
    #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()
    sleep(2)
    driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
    sleep(5)
    

    #--------------------------------------------------------------- ok

    lista_actores_peli_i=[]
    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,12):

        #accedemos a la informacion del actor/actriz:
        try:
            driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
            sleep(5)                    
            #-----------------------------------------------------------------ok

            #seleccionamos MENU desplegable, para acceder a biografia y premios:
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
            sleep(2)
            #acceso a la biografia
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
            sleep(2)
            #----------------------------------------------------------------ok
            
            # obtenemos el NOMBRE del actor
            nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
            sleep(5)
            #---------------------------------------------------------------- ok
            #obtenemos AÑO
            try:
                año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
                año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
            except:
                año='NULL'
            sleep(3)

            #---------------------------------------------------------------- ok

            #obtenemos el texto para filtrar informacion 'CONOCIDO POR':
            try:
                # scroll hasta el texto que necesitamos
                xpath_d='//*[@id="mini_bio"]'
                pyhton_d= driver.find_element(By.XPATH, xpath_d)
                driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
                sleep(6)
                #Obtenemos el texto:
                driver.find_element("xpath", '//*[@id="mini_bio"]').click()
                texto_actor = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text
                
                sleep(6)
                conocido_por=[]
                if re.search('conocido por|conocida por', texto_actor) == None:
                        conocido_por='NULL'
                else:
                    patron_a='\.'
                    patron_b='conocido por|conocida por' # siempre me va a dar un len()=2, por que en la cadena solo hay un 'conocido por' y este siempre estara en [1]
                    patron_c='\(\d{4}'
                    patron_d="\), |\) y |\)"
                    lista_sep=re.split(patron_b, texto_actor)
                    lista_sep_2=re.split(patron_a, lista_sep[1])
                    # la estring que nos interesa termina con un '.' simpre va a estar en [0] por que es consecutivo al 'conocido por'
                    lista_sep_3=re.split(patron_c, lista_sep_2[0])
                        
                    for y in lista_sep_3:
                        lista_sep_3_sin_espacios=(re.sub(patron_d,'', y)).strip() #quitamos espacios y parentesis
                        if lista_sep_3_sin_espacios != '': #quitamos de la lista posibles espacios
                            conocido_por.append(lista_sep_3_sin_espacios)
            except:
                conocido_por='NULL'
            #---------------------------------------------------------------- ok
            sleep(2)
            driver.back() # al acceder a 'mi biografía' hacemos click. tenemos que volver atrás. SEGUIMOS DENTRO DE BIOGRAFIA ACTOR
            sleep(2)
            driver.back() # VOLVEMOS A PAGINA PRINCIPAL ACTOR.
            sleep(10)

            #--------------------------------------------------------------- ok
            #acceso a los PREMIOS, desde el desplegable.
            try:
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
            
            except:
                #si se sale de la pagina, (si no hay texto, conocido por) volvemos a ella:
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click() #acceso actor.
                sleep(2) 
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a los premios
                sleep(2)

    
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la premios.
            sleep(6)
            #creamos ciclo for para sacar las los premios:
            
            try:
                premios=[]
                for i in range(1,6): #solo queremos 5 pelis
                    premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                    premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                    premios.append(premio_limpio)
                sleep(6)
            except:
                premios='NULL'
            
            

            driver.back()
            sleep(3)

            #--------------------------------------------------------------- ok
            #extraemos informacion que hace.

            que_hace=[]
            try:
                try:
                    for z in range(1,4):
                        peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                        que_hace.append(peli_hace)
                        sleep(2)
                except:
                        peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                        que_hace.append(peli_hace)
                        sleep(2)
            except:
                que_hace='NULL'
            
            #--------------------------------------------------------------- ok

            #sacamos tupla de cada actor
            lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
            tupla_x=tuple(lista_infor_actor)
            lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula
        
            sleep(6)
            driver.back()
            #volvemos a la página principal 
        except:
            pass

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [6]:
datos_csv =pd.read_csv(r"C:\Users\marg_\Desktop\Proyecto_lesion2\project-da-promo-H-module-2-team-Interstellar\SELENIUM\sublista_selenium.csv") 

In [7]:
datos_csv


,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [8]:
id_ = datos_csv["ID_Pelicula"]

In [9]:
lista_id = id_.to_list()
print(id_)


0       tt1800301
1       tt1847719
2      tt10270648
3      tt14991492
4       tt2363363
          ...    
995    tt10106152
996    tt10363182
997     tt3857098
998    tt10095498
999     tt1067107
Name: ID_Pelicula, Length: 1000, dtype: object


In [10]:
#parte 1
lista_id_parte_1= lista_id[0:101]

In [11]:
dic_peli={'id_peli':[], 'actores_10':[]}
no_peli=[]
for id_peli in tqdm(lista_id_parte_1):
    try:
        tupla_una_peli=tuplas_actores(id_peli)
        print(tupla_una_peli)
        dic_peli['actores_10'].append(tupla_una_peli)
        dic_peli['id_peli'].append(id_peli)
    except:
        no_peli.append(id_peli)
        print('fallo peli:--------->', id_peli)
        continue

    
print(dic_peli)

  1%|          | 1/101 [12:26<20:44:52, 746.93s/it]

[('Al Bianchi', 'NULL', ['Reparto', 'Dirección de casting', 'Reparto'], ['su papel en Yo, Tonya', 'The Red, Black and Blonde: God $ave', 'Velma Is Living'], 'NULL'), ('Thomas Brown', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Gerry Cheney', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave', 'Corruption'], 'NULL'), ('Rabbit Duran', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Chelsey Green', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave', 'Bed Time Ghost Tales'], 'NULL'), ('Kevin Grogen', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Elizabeth Hoots', 'NULL', ['Reparto', 'Cámara y electricidad', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave', 'Wet Behind the Ears', 'Bleed'], 'NULL'), ('Laurence Love', 'NULL', ['Sonido', 'Reparto', 'Sonido'], ['su papel

  2%|▏         | 2/101 [23:00<18:42:43, 680.44s/it]

[('Dennis Haladyn', 'NULL', ['Reparto', 'Especialistas', 'Guion'], ['su papel en Tell Me Your Secret', 'Enter the Cowboy', 'Westbrick Murders'], ['Action on Film International Film Festival, USA 2017 Nominado Action on Film Award Best Foreign Film - Short Enter the Cowboy Compartido con: Jack Hansen 2017 Nominado Action on Film Award Best Western Enter the Cowboy Compartido con: Jack Hansen 2012 Nominado Action on Film Award Best Supporting Actor Små mænd', 'CPH PIX 2016 Ganador Movie Battle The Shoot Out Award Enter the Cowboy Compartido con: Jack Hansen 2012 Nominado Movie Battle Best Acting Performance Supernatural Tales', 'Berlin Short Film Festival 2017 Ganador Jury Prize Best Comedy Short Film Enter the Cowboy Compartido con: Jack Hansen', '60 Seconds Festival 2014 Ganador Audience Award Tell Me Your Secret Compartido con: Daniel Urhøj', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información ac

  3%|▎         | 3/101 [34:20<18:30:39, 679.99s/it]

...

[('Jirí Langmajer', '1966', ['Reparto', 'Banda sonora', 'Reparto'], ['El idiota', 'Ordinace v ruzové zahrade', 'Pravidla lzi'], 'NULL'), ('Ivana Chýlková', '1963', ['Reparto', 'Banda sonora', 'Reparto'], ['Díky za kazdé nové ráno', 'Cas sluhu', 'Prítelkyne z domu smutku'], 'NULL'), ('Lucie Zácková', '1978', ['Reparto', 'Reparto'], ['Kobry a uzovky', 'Karamazovi', 'Rapl'], 'NULL'), ('Iva Janzurová', '1941', ['Reparto', 'Banda sonora', 'Reparto'], ['Ene bene', 'Výlet', 'Co chytnes v zite'], ["Art Film Festival 1999 Ganador Actor's Mission Award", 'Czech Lions 2020 Nominado Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Teroristka 2003 Ganador Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Výlet 2001 Nominado Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Ene bene 1999 Ganador Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Co chytnes v zite', "Karlovy Vary International Film Festival 2015 Ganador Festival President's Award For outsta

 50%|█████     | 51/101 [7:15:32<9:28:21, 682.03s/it]

[('Ace Malik', 'NULL', ['Reparto', 'Reparto'], ['su papel en Nightfall: Second World III', 'Second World: Blood, Fire and Smoke'], 'NULL'), ('Altorro Prince Black', 'NULL', ['Reparto', 'Dirección', 'Casting'], ['su papel en Boundary', 'House of Cards', 'City of Lost Souls'], ["LA Web Fest 2011 Ganador Outstanding Achievement Award Outstanding Lead Actor In A Mockumentary Series Sista Wives: Jacob's House", "World Music and Independent Film Festival 2015 Nominado Festival Award Best Actor - Feature Film Bachelorette's Degree 2015 Nominado Festival Award Best Director - Feature Film Bachelorette's Degree 2013 Ganador Festival Award Best Supporting Actor in a Short Film / DMV Table for Three 2013 Nominado Festival Award Best Supporting Actor in a Feature / DMV Dead Reckoning", 'Television, Internet and Video Association of DC (TIVA-DC) Peer Awards 2016 Ganador Peer Award Acting on Camera - Fiction, Male City of Lost Souls 2016 Ganador Peer Award Acting on Camera - Fiction, Male Reunion', 

 51%|█████▏    | 52/101 [7:22:09<8:07:18, 596.70s/it]

[('Bianca Boyd', 'NULL', ['Reparto', 'Reparto'], ['su papel en Standing Up for Sunny', 'Rip Tide', 'The Secret Letters Project'], 'NULL'), ('Sophie Lowe', 'NULL', ['Reparto', 'Edición', 'Producción'], ['Derribad al hombre', 'Medieval', 'The Dive'], ['Film Critics Circle of Australia Awards 2010 Nominado FCCA Award Best Supporting Actor - Female Beautiful Kate', 'Australian Film Institute 2009 Nominado AFI Award Best Lead Actress Beautiful Kate', 'Screen Music Awards, Australia 2010 Nominado Screen Music Award Best Original Song Composed for a Feature Film, Telemovie, TV Series or Mini-Series Road Train Compartido con: Rafael May', 'The Equity Ensemble Awards 2012 Ganador The Equity Award Outstanding Performance by an Ensemble in a Mini-series or Telemovie The Slap Compartido con: Raffaele Costabile · Blake Davis · Essie Davis · Alex Dimitriades · Melissa George · Diana Glenn · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guí

In [1]:
 [('Al Bianchi', 'NULL', ['Reparto', 'Dirección de casting', 'Reparto'], ['su papel en Yo, Tonya', 'The Red, Black and Blonde: God $ave', 'Velma Is Living'], 'NULL'), ('Thomas Brown', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Gerry Cheney', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave', 'Corruption'], 'NULL'), ('Rabbit Duran', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Chelsey Green', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave', 'Bed Time Ghost Tales'], 'NULL'), ('Kevin Grogen', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Elizabeth Hoots', 'NULL', ['Reparto', 'Cámara y electricidad', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave', 'Wet Behind the Ears', 'Bleed'], 'NULL'), ('Laurence Love', 'NULL', ['Sonido', 'Reparto', 'Sonido'], ['su papel en The Red, Black and Blonde: God $ave', 'Billy Bates'], 'NULL'), ('Josh Miley', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Red, Black and Blonde: God $ave'], 'NULL'), ('Russel Minton', 'NULL', ['Reparto', 'Reparto'], ['su papel en Between Kings and Queens', 'The Red, Black and Blonde: God $ave', 'The Walk with Darkness'], 'NULL')]
  ,[('Dennis Haladyn', 'NULL', ['Reparto', 'Especialistas', 'Guion'], ['su papel en Tell Me Your Secret', 'Enter the Cowboy', 'Westbrick Murders'], ['Action on Film International Film Festival, USA 2017 Nominado Action on Film Award Best Foreign Film - Short Enter the Cowboy Compartido con: Jack Hansen 2017 Nominado Action on Film Award Best Western Enter the Cowboy Compartido con: Jack Hansen 2012 Nominado Action on Film Award Best Supporting Actor Små mænd', 'CPH PIX 2016 Ganador Movie Battle The Shoot Out Award Enter the Cowboy Compartido con: Jack Hansen 2012 Nominado Movie Battle Best Acting Performance Supernatural Tales', 'Berlin Short Film Festival 2017 Ganador Jury Prize Best Comedy Short Film Enter the Cowboy Compartido con: Jack Hansen', '60 Seconds Festival 2014 Ganador Audience Award Tell Me Your Secret Compartido con: Daniel Urhøj', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('David Sakurai', 'NULL', ['Reparto', 'Producción', 'Guion'], ['Animales fantásticos: Los crímenes de Grindelwald', 'Avatar: La leyenda de Aang', 'El hombre en el castillo'], ['Action on Film International Film Festival, USA 2014 Nominado Action on Film Award Male Action Performer of the Year Echoes of a Ronin 2014 Nominado Action on Film Award Breakout Action Star - Male - Short Echoes of a Ronin 2010 Ganador Action on Film Award Breakout Action Star - Male - Short Eastern Army', 'CPH PIX 2012 Nominado Movie Battle Best Acting Performance Supernatural Tales 2010 Ganador Movie Battle Best Actor Eastern Army', 'Los Angeles New Wave International Film Festival, US 2011 Ganador Short Film Award Best Stunts Eastern Army Compartido con: Philip Th. Pedersen', 'American International Film Festival, San Gabriel 2010 Nominado AIFF Award Best Actor Eastern Army', 'Nightmares Film Festival 2017 Nominado Nightmares Film Festival Award Best Actor Feature Le Accelerator']), ('Daniell Edwards', '1980', ['Reparto', 'Especialistas', 'Producción'], ['Ud af mørket', 'Westbrick Murders', 'Pistoleros'], 'NULL'), ('Maja Muhlack', '1977', ['Reparto', 'Producción', 'Reparto'], ['Westbrick Murders', 'One Hell of a Christmas', 'The Horror Vault'], 'NULL'), ('Hector Vega Mauricio', '1965', ['Reparto', 'Producción', 'Dirección'], ['Supernatural Tales', 'Pistoleros', 'One Hell of a Christmas'], 'NULL'), ('Ted Pappas', '1953', ['Reparto', 'Reparto'], ['The Last Demon Slayer', 'Cuando un hombre vuelve a casa', 'Frihed på prøve'], 'NULL'), ('Erik Holmey', '1942', ['Reparto', 'Reparto'], ['Conan, el bárbaro', 'One Hell of a Christmas', 'Pistoleros'], 'NULL'), ('Joon Poore', 'NULL', ['Especialistas', 'Reparto', 'Edición'], ['su papel en Shrouded Destiny: A Star Wars Long Tale'], 'NULL')]
  3%|▎         | 3/101 [34:20<18:30:39, 679.99s/it][('London Albert', 'NULL', ['Reparto', 'Guion', 'Edición'], ['su papel en Martinelli a Star Wars Fan Story'], 'NULL'), ('Jaleah Ivory', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Martinelli a Star Wars Fan Story'], 'NULL'), ('Jamya Ivory', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Martinelli a Star Wars Fan Story'], 'NULL'), ('Erik Martinelli', 'NULL', ['Reparto', 'Reparto'], ['su papel en Martinelli a Star Wars Fan Story', 'Undead Radio'], 'NULL'), ('Michelle Martinelli', 'NULL', ['Reparto', 'Reparto'], ['su papel en Martinelli a Star Wars Fan Story', 'Undead Radio'], 'NULL'), ('Vincent Martinelli', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en The Adventures of Jackson Silo, Messy the Cat: The Movie y Martinelli a Star Wars Fan Story'], 'NULL'), ('Jessica Williams', 'NULL', ['Reparto', 'Reparto'], ['su papel en Martinelli a Star Wars Fan Story'], 'NULL'), ('Joshua Williams', 'NULL', ['Dirección', 'Reparto', 'Guion'], ['su papel en Judge Jhorts', 'Dr'], 'NULL'), ('Mark Williams', 'NULL', ['Reparto', 'Edición', 'Producción'], ['su papel en The Adventures of Jackson Silo y Martinelli a Star Wars Fan Story'], 'NULL')]
  4%|▍         | 4/101 [46:58<19:09:06, 710.79s/it][('Fernando Rocha', '1975', ['Reparto', 'Guion', 'Reparto'], ['Balas & Bolinhos: O Último Capítulo', 'Maré Alta', 'Balas & Bolinhos - O Regresso'], 'NULL'), ('João Seabra', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en 2 Duros de Roer', 'Balas & Bolinhos: O Último Capítulo', 'Do Outro Lado do Tejo'], 'NULL'), ('Mafalda Luís de Castro', '1989', ['Reparto', 'Reparto'], ['Lua Vermelha', 'Ana y los siete', '2 Duros de Roer'], 'NULL'), ('Melânia Gomes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Maré Alta', 'A Impostora', 'Deixa Que Te Leve'], 'NULL'), ('Susana Cacela', 'NULL', ['Reparto', 'Reparto'], ['su papel en Maré Alta', 'Mar Salgado', '2 Duros de Roer'], 'NULL'), ('Jorge Mourato', '1974', ['Reparto', 'Reparto'], ['Bem-Vindos a Beirais', 'Laços de Sangue', 'País Irmão'], 'NULL'), ('Aldo Lima', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Conexão', 'Arte de Roubar', 'Miss Beijo'], 'NULL'), ('Francisco Menezes', '1973', ['Reparto', 'Guion', 'Producción'], ['Bad Investigate', 'Balas & Bolinhos: O Último Capítulo', '2 Duros de Roer'], 'NULL'), ('Alexandre Santos', 'NULL', ['Reparto', 'Reparto'], ['su papel en 2 Duros de Roer'], 'NULL'), ('Joaquim Guerreiro', '1966', ['Reparto', 'Guion', 'Reparto'], ['Mutant Blast', 'Era Uma Vez'], 'NULL')]
  5%|▍         | 5/101 [53:32<15:54:39, 596.66s/it][('Ken Duken', '1979', ['Reparto', 'Producción', 'Dirección'], ['Max Manus', 'Malditos bastardos', 'Destino: La saga Winx'], ['Grimme Award 2008 Ganador Adolf Grimme Award Fiction En otra Liga Compartido con: Buket Alakus · Jan Berger · Karoline Herfurth · Thierry van Werveke 2005 Ganador Adolf Grimme Award Fiction/Entertainment Kiss and Run Compartido con: Maggie Peren · Annette Ernst', 'Bavarian TV Awards 2017 Nominado Bavarian TV Award Best Actor in a Movie Made for Television (Bester Schauspieler - Fernsehspiel/Fernsehfilm) Tempel 2009 Ganador Bavarian TV Award Best Actor in a Movie Made for Television (Bester Schauspieler - Fernsehspiel/Fernsehfilm) Willkommen zuhause', 'German Television Awards 2017 Nominado German Television Award Best Actor (Bester Schauspieler) TempelDuelo de hermanos: La historia de Adidas y Puma', 'Max Ophüls Festival 2005 Nominado Best Short Film From Another Point of View Compartido con: Bernd Katzmarczyk', 'Monte-Carlo TV Festival 2011 Nominado Golden Nymph Mini-Series - Best Performance by an Actor Laconia, el hundimiento']), ('Dagny Dewath', '1981', ['Reparto', 'Reparto'], ['Sturm der Liebe', 'Robin Hood', 'Professor T'], 'NULL'), ('Matthias Koeberlin', '1974', ['Reparto', 'Banda sonora', 'Reparto'], ['Tornado - La furia del cielo', 'Fugitivos', 'Lutter'], ['Bavarian TV Awards 2011 Nominado Bavarian TV Award Best Actor in a Movie Made for Television (Bester Schauspieler - Fernsehspiel/Fernsehfilm) Restrisiko', 'German Television Awards 2007 Ganador German Television Award Best Actor (Bester Schauspieler) Tornado - La furia del cielo', 'Günter Strack TV Award 2000 Ganador Günter Strack TV Award Best Young Actor Ben & Maria - Liebe auf den zweiten Blick', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Stipe Erceg', '1974', ['Reparto', 'Producción', 'Reparto'], ['Los edukadores', 'Sin identidad', 'RAF Facción del ejército rojo'], ["Grimme Award 2013 Ganador Audience Award of the 'Marl Group' Blaubeerblau Compartido con: Rainer Kaufmann · Beate Langmaack · Devid Striesow", 'Max Ophüls Festival 2004 Ganador Best Young Actor Yugotrip', 'Munich Film Festival 2004 Ganador Young German Cinema Award Acting - Male (Schauspiel - männlich) Such mich nichtLos edukadores', 'Hessian TV Award 2012 Ganador Hessian TV Award Best Actor (Bester Darsteller) Blaubeerblau', 'New Faces Awards, Germany 2005 Nominado New Faces Award Actor Los edukadores'])]
  6%|▌         | 6/101 [53:56<10:36:18, 401.88s/it][]
  7%|▋         | 7/101 [1:01:39<11:00:46, 421.77s/it][('Anmol K.C.', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Dreams', 'Super Star', 'Karodpati'], 'NULL'), ('Resha Ale Magar', 'NULL', ['Reparto', 'Reparto'], ['su papel en Chhadke 2'], 'NULL'), ('Santosh Mizar', 'NULL', ['Reparto', 'Reparto'], ['su papel en Chhadke 2'], 'NULL'), ('Kamal Mani Nepal', 'NULL', ['Reparto', 'Reparto'], ['su papel en Taandro', 'Jatra', 'Kabaddi'], 'NULL'), ('Pradip Raut', '2004', ['Reparto', 'Reparto'], ['Chhadke 2'], 'NULL'), ('Robin Tamang', '1963', ['Reparto', 'Música', 'Banda sonora'], ['Chhadke 2'], 'NULL')]
  8%|▊         | 8/101 [1:13:59<13:31:10, 523.34s/it][('Davin Broadhurst', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone', 'Sunshine & Rainbows', 'The Red Canary'], 'NULL'), ('Brandon Criswell', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone'], 'NULL'), ('Evan Barnard', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Spider-Man: Hunted', 'Payphone', 'Darryl vs'], 'NULL'), ('Christopher Sabolcik', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone'], 'NULL'), ('Collins Cornwell', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone'], 'NULL'), ('Jayden Peszko', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone', 'Dingus', 'Three'], 'NULL'), ('Nifa Aulo', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone'], 'NULL'), ('Justin Cho', 'NULL', ['Dirección', 'Dirección de fotografía y cámara', 'Edición'], ['su papel en Payphone', 'Three'], 'NULL'), ('Charlie McVickers', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone'], 'NULL'), ('Kaylie Beard', 'NULL', ['Reparto', 'Reparto'], ['su papel en Payphone'], 'NULL')]
  9%|▉         | 9/101 [1:18:07<11:10:14, 437.11s/it][('Sky Sengnaryvong', 'NULL', ['Reparto', 'Reparto'], ['John Lick My Nuts', 'A Knives Day Out', 'Cult Wars: Episode I - The Fandom Menace'], 'NULL'), ('Francis Brissette', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Cult Wars: Episode I - The Fandom Menace', 'Tony the Toxic Owl: Toxic Shorts', 'John Lick My Nuts'], 'NULL'), ('Jerry Werdann', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Tony the Toxic Owl: Toxic Shorts', 'Cult Wars: Episode I - The Fandom Menace', 'John Lick My Nuts'], 'NULL')]
 10%|▉         | 10/101 [1:31:28<13:53:30, 549.56s/it][('Anaïs Demoustier', '1987', ['Reparto', 'Música', 'Banda sonora'], ["L'enfance du mal", 'Las nieves del Kilimanjaro', 'Los consejos de Alice'], ['Berlin International Film Festival 2010 Ganador EFP Shooting Star France.', 'César Awards, France 2023 Nominado César Best Supporting Actress (Meilleure actrice dans un second rôle) Novembre 2020 Ganador César Best Actress (Meilleure actrice) Los consejos de Alice 2018 Nominado César Best Supporting Actress (Meilleure actrice dans un second rôle) La casa junto al mar 2011 Nominado César Most Promising Actress (Meilleur espoir féminin) De amor y de agua fresca 2009 Nominado César Most Promising Actress (Meilleur espoir féminin) Les grandes personnes', "Karlovy Vary International Film Festival 2010 Ganador Best Actress L'enfance du mal", 'SACD Awards 2011 Ganador Suzanne Bianchetti Award', 'Lumiere Awards, France 2020 Nominado Lumiere Award Best Actress (Meilleure actrice) Los consejos de Alice']), ('Josiane Balasko', '1950', ['Reparto', 'Guion', 'Dirección'], ['Felpudo maldito', 'El quimérico inquilino', 'La clienta'], ['César Awards, France 2021 Ganador Anniversary César 2020 Nominado César Best Supporting Actress (Meilleure actrice dans un second rôle) Gracias a Dios 2004 Nominado César Best Actress (Meilleure actrice) Cette femme-là 2000 Ganador Honorary César 1996 Nominado César Best Director (Meilleur réalisateur) Felpudo maldito 4 más', 'Palm Springs International Film Festival 1996 Ganador Audience Award Best Foreign Language Film Felpudo maldito', 'Sundance Film Festival 2009 Nominado Grand Jury Prize World Cinema - Dramatic La clienta', 'Lumiere Awards, France 1996 Ganador Lumiere Award Best Screenplay (Meilleur scénario) Felpudo maldito', "Rome Film Fest 2008 Nominado Golden Marc'Aurelio Award La clienta"]), ('Philippe Katerine', '1968', ['Reparto', 'Composición', 'Guion'], ['El gran baño', 'La verdad sobre Charlie', 'Gainsbourg (Vida de un héroe'], 'NULL'), ('William Lebghil', '1990', ['Reparto', 'Banda sonora', 'Reparto'], ['Mentes brillantes', "C'est la vie", 'Yves'], ['César Awards, France 2019 Nominado César Most Promising Actor (Meilleur espoir masculin) Mentes brillantes', 'Lumiere Awards, France 2019 Nominado Lumiere Award Most Promising Actor (Meilleure révélation masculine) Mentes brillantes', 'Club Média Ciné 2019 Ganador Breakthrough Actress (Prix Révélation Féminine) Mentes brillantes', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Sergi López', '1965', ['Reparto', 'Banda sonora', 'Reparto'], ['El laberinto del fauno', 'Negocios ocultos', 'Harry, un amigo que os quiere'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2007 Nominado Saturn Award Best Supporting Actor El laberinto del fauno', "Acteurs à l'Écran 1993 Ganador Prix Michel Simon Best Actor La petite amie d'Antonio", 'Amiens International Film Festival 2015 Ganador Best Actor Un día perfecto para volar', 'Sitges - Catalonian International Film Festival 1997 Ganador Gran Angular Award Best Actor Western', 'César Awards, France 2001 Ganador César Best Actor (Meilleur acteur) Harry, un amigo que os quiere 1998 Nominado César Most Promising Actor (Meilleur jeune espoir masculin) Western']), ('Philippe Duquesne', '1965', ['Reparto', 'Banda sonora', 'Reparto'], ['Largo domingo de noviazgo', 'Bienvenidos al Norte', 'La verdad sobre Charlie'], 'NULL'), ('Olivier Broche', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en La verdad sobre Charlie', 'El instante más oscuro', 'Chez Maupassant'], 'NULL'), ('Jocelyne Augier', 'NULL', ['Reparto', 'Reparto'], ['su papel en La pièce rapportée', 'Yann'], 'NULL'), ('Christian Taponard', 'NULL', ['Reparto', 'Reparto'], ["su papel en Le retour d'Arsène Lupin", 'El pequeño Nicolás', 'Pour une femme'], 'NULL'), ('Olivier Degorgue', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Impact', 'Enfers y Interpol'], 'NULL')]
 11%|█         | 11/101 [1:42:48<14:43:55, 589.29s/it][('Raj Singh Jhinjar', 'NULL', ['Reparto', 'Guion', 'Reparto'], 'NULL', 'NULL'), ('Himanshi Parashar', '2000', ['Reparto', 'Reparto'], ['Teri Meri Doriyaann', 'Babbar', 'Gedi Route'], 'NULL'), ('Gurdeep Manalia', 'NULL', ['Guion', 'Reparto', 'Guion'], ['su papel en Dreamland', 'Babbar', 'Furteela'], 'NULL'), ('Seven Chahal', 'NULL', ['Reparto', 'Reparto'], ['su papel en Nishana', 'Warning 2', 'Babbar'], 'NULL'), ('Surinder Arora', 'NULL', ['Reparto', 'Reparto'], ['su papel en U Turn', 'Babbar', 'A Tale of Punjab'], 'NULL'), ('Raghveer Boli', '1987', ['Reparto', 'Reparto'], ['Warning 2', 'Sheran Di Kaum Punjabi', 'Ik Sandhu Hunda Si'], 'NULL'), ('Raaf Chaudhary', 'NULL', ['Reparto', 'Música', 'Reparto'], ['su papel en Bodyguard', 'Airlift', 'Tiger está vivo'], 'NULL'), ('Pardeep Cheema', 'NULL', ['Reparto', 'Reparto'], ['su papel en Warning 2', 'Yug the law of karma', 'Niebla de invierno'], 'NULL'), ('Ravi Dehru', 'NULL', ['Reparto', 'Edición', 'Dirección'], 'NULL', 'NULL')]
 12%|█▏        | 12/101 [1:43:12<10:19:20, 417.53s/it][]
 13%|█▎        | 13/101 [1:50:53<10:31:26, 430.53s/it][('Tess Spentzos', 'NULL', ['Reparto', 'Dirección de fotografía y cámara', 'Equipos adicionales'], ['su papel en The Light Touch', 'Saw IV', 'Lurk'], ['Maverick Movie Awards 2019 Nominado Maverick Movie Award Best Actress: Feature The Light Touch', 'Toronto Film Magazine Fest 2021 Ganador March Award Best Actress The Light Touch', 'Luleå International Film Festival 2021 Ganador Best Actress in a Feature The Light Touch', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Peter Gerald', 'NULL', ['Reparto', 'Reparto'], ['su papel en Lurk', 'Malefaction'], 'NULL'), ('Aris Athan', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Lurk', 'Celebrating Life', 'December Riots'], 'NULL'), ('Tina Leonora', 'NULL', ['Reparto', 'Reparto'], ['su papel en Lurk', 'I ekdromi', 'The Shadows Inside Me'], 'NULL'), ('Maria Mihailidou', 'NULL', ['Reparto', 'Reparto'], ['su papel en Lurk'], 'NULL'), ('Sofia Spyridou', 'NULL', ['Reparto', 'Reparto'], ['su papel en Lurk'], 'NULL')]
 14%|█▍        | 14/101 [2:03:54<12:57:42, 536.35s/it][('Salomon Wise', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en Projekt Adler - Ein letzter Job'], 'NULL'), ('Andreas Nguyen', 'NULL', ['Especialistas', 'Reparto', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en Luc Van Tien: Tuyet Dinh Kungfu', 'Hitman: Agente 47', 'Liger'], ['Filmfare Awards 2021 Nominado Technical Award Best Action Khuda Haafiz Compartido con: Viktor Ivanov 2020 Nominado Technical Award Best Action Commando 3 Compartido con: Dawid Szatarski · Allan Amin · K. Ravivarma', 'Action on Film International Film Festival, USA 2018 Ganador Best Picture Best Picture Luc Van Tien: Tuyet Dinh Kungfu 2018 Ganador Best Action Sequence - Martial Arts Best Action Sequence Martial Arts - Feature Luc Van Tien: Tuyet Dinh Kungfu 2018 Nominado Best Fight Choreography - Feature Best Fight Choreography - Feature Luc Van Tien: Tuyet Dinh Kungfu 2018 Nominado Best Drama Best Drama Luc Van Tien: Tuyet Dinh Kungfu 2018 Nominado Best Foreign Film Best Foreign Film Luc Van Tien: Tuyet Dinh Kungfu 1 más', 'German Television Academy Awards 2014 Nominado German Television Academy Award Best Stunt (Bester Stunt) Nachbarn süß-sauer', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Bela Klentze', '1988', ['Reparto', 'Reparto'], ['The Lost Picture, En un lugar de África', 'Factor 8'], 'NULL'), ('Andreas Pape', '1980', ['Reparto', 'Dirección de fotografía y cámara', 'Cámara y electricidad'], ['La petite mort', 'Sounds of Fear', 'M III: Century of the Dead'], 'NULL'), ('Curd Berger', 'NULL', ['Reparto', 'Reparto'], ['su papel en Breakdown Forest - Reise in den Abgrund', 'Projekt Adler - Ein letzter Job', '25 km/h'], 'NULL'), ('Viktor Krüger', 'NULL', ['Reparto', 'Especialistas', 'Guion'], ['su papel en Luden', 'En el lugar del crimen', 'Zero woman: Kesenai kioku'], 'NULL'), ('Felix Fukuyoshi Ruwwe', 'NULL', ['Especialistas', 'Reparto', 'Cámara y electricidad'], ['su papel en Duty - Out of Line', 'Matrix Resurrections', 'Los juegos del hambre: Balada de pájaros cantores y serpientes'], ['Screen Actors Guild Awards 2022 Nominado Actor Outstanding Action Performance by a Stunt Ensemble in a Motion Picture Matrix Resurrections Compartido con: Joel Adrian · Steve Albrecht · Abdulmoumen Ali · Christian Angres · Dave Ardito · Raffael Armbruster · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Bloodstained Indie Film Festival 2021 Ganador Festival Award Best Action Short Duty - Out of Line Compartido con: Julia Schunevitsch', 'The Yellow Fever Independent Film Festival 2021 Ganador Best Action Award Best Action Duty - Out of Line Compartido con: Julia Schunevitsch 2021 Ganador Graeme Livingstone Award Most Memorable Movie Duty - Out of Line Compartido con: Julia Schunevitsch 2021 Nominado Best Short Film Award Best Short Film Duty - Out of Line Compartido con: Julia Schunevitsch', 'Mid West Action fest 2021 Ganador Best Action Sequence Best Action Sequence Duty - Out of Line', 'Fighting Spirit Film Festival 2021 Ganador Audience Award Audience Choice Duty - Out of Line 2021 Nominado Festival Award Best International Short Film Duty - Out of Line']), ('Daniele Balconi', 'NULL', ['Especialistas', 'Reparto', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en Liger', 'Lift: Un robo de primera clase', 'The Equalizer 3'], 'NULL'), ('Sefa Demirbas', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en Dune: Parte dos', 'Rayo Murali', 'Sanak'], 'NULL'), ('Tanguy Guinchard', 'NULL', ['Especialistas', 'Reparto', 'Dirección'], ['su papel en Los juegos del hambre: Balada de pájaros cantores y serpientes', 'Buon Appetito', 'Underwater'], ['Cannes Film Awards 2024 Nominado Honorable Mention Best Short Film Buon Appetito 2024 Ganador Cannes Film Awards Trophy Best Fight Choreography Buon Appetito', 'Malta Film Awards 2024 Ganador Malta Film Award Best Action Buon Appetito Awarded for Best Action film Malta film award 2024', '8 And HalFilm Awards 2022 Ganador 8 & HalFilm Award - October Best Cinematography Underwater', 'Cine Paris Film Festival 2022 Ganador October Award Best Cinematography Underwater', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir'])]
 15%|█▍        | 15/101 [2:17:36<14:52:08, 622.43s/it][('Luke Goss', '1968', ['Reparto', 'Producción', 'Dirección'], ['Hellboy II: El ejército dorado', 'Blade II', 'After the Screaming Stops'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2002 Nominado Cinescape Genre Face of the Future Award Male Blade II', 'MTV Movie + TV Awards 2009 Nominado MTV Movie Award Best Villain Hellboy II: El ejército dorado 2009 Nominado MTV Movie Award Best Fight Hellboy II: El ejército dorado Compartido con: Ron Perlman', 'Teen Choice Awards 2008 Nominado Teen Choice Award Choice Movie Rumble Hellboy II: El ejército dorado Compartido con: Ron Perlman', 'Character and Morality in Entertainment Awards 2007 Ganador Camie Actor Una noche con el rey', 'East Europe International Film Festival 2020 Nominado Fusion Award Best Lead Actor The Loss Adjuster']), ('Heather Roop', '1976', ['Reparto', 'Producción', 'Reparto'], ['Operación AWOL-72', 'The Sacred', 'All My Children'], 'NULL'), ('RZA', '1969', ['Reparto', 'Guion', 'Composición'], ['El hombre de los puños de hierro', 'American Gangster', 'Kill Bill: Volumen 1'], ['BAFTA Awards 2004 Nominado Anthony Asquith Award for Film Music Kill Bill: Volumen 1', 'Primetime Emmy Awards 2020 Nominado Primetime Emmy Outstanding Original Main Title Theme Music Wu-Tang: An American Saga', 'International Documentary Association 2019 Nominado IDA Award Best Music Documentary Wu-Tang Clan. Revolución Hip hop Compartido con: Sacha Jenkins · Peter J. Scalettar · Peter Bittenbender · Vinnie Malhotra · Cary Graber · Vanessa Maruskin · · · · · · · · ·', 'Online Film Critics Society Awards 2004 Nominado OFCS Award Best Original Score Kill Bill: Volumen 1', 'Screen Actors Guild Awards 2008 Nominado Actor Outstanding Performance by a Cast in a Motion Picture American Gangster Compartido con: Armand Assante · Josh Brolin · Russell Crowe · Ruby Dee · Chiwetel Ejiofor · Idris Elba · Cuba Gooding Jr. · · · · · · · ·']), ('Bokeem Woodbine', '1973', ['Reparto', 'Producción', 'Banda sonora'], ['Spider-Man: Homecoming', 'Fargo', 'Queen & Slim'], ['Primetime Emmy Awards 2016 Nominado Primetime Emmy Outstanding Supporting Actor in a Limited Series or a Movie Fargo For playing: "Mike Milligan".', 'Image Awards (NAACP) 2001 Nominado Image Award Outstanding Supporting Actor in a Drama Series City of Angels', 'Black Reel Awards 2023 Nominado Black Reel Outstanding Supporting Actor La inspección 2016 Ganador Black Reel Outstanding Supporting Actor, TV Movie or Limited Series Fargo 2015 Nominado Black Reel Outstanding Supporting Actor, TV Movie or Mini-Series The Fright Night Files', "Critics Choice Television Awards 2016 Nominado Critics' Choice TV Award Best Supporting Actor in a Movie Made for Television or Limited Series Fargo", 'Gold Derby Awards 2019 Nominado Gold Derby TV Award TV Movie/Mini Supporting Actor of the Decade Fargo The People v. O.J. Simpson 2016 Nominado Gold Derby TV Award Breakthrough Performer of the Year 2016 Nominado Gold Derby TV Award TV Movie/Mini Supporting Actor Fargo']), ('Louis Mandylor', '1966', ['Reparto', 'Producción', 'Dirección'], ['Friends', 'Rambo: Last Blood', 'La deuda'], ['New York International Independent Film & Video Festival 2003 Ganador Feature Film Award Best Actor Fiebre blanca', 'Action on Film International Film Festival, USA 2017 Nominado Action on Film Award Male Action Performer of the Year California Dreaming 2013 Nominado Action on Film Award Best Supporting Actor - Feature 20 Feet Below: The Darkness Descending Compartido con: Kristoff St. John · Danny Trejo 2010 Ganador Action on Film Award Best Supporting Actor - Short Stained 2009 Ganador Action on Film Award Best Motion Picture - Drama In the Eyes of a Killer Compartido con: Mamie Jean Calvert 2009 Ganador AOF/WAB Award Best Actor in a Leading Role In the Eyes of a Killer 1 más', 'Sacramento International Film Festival 2009 Ganador Festival Prize Outstanding Achievement in Feature Film In the Eyes of a Killer Compartido con: Mamie Jean Calvert 2009 Ganador Festival Prize Outstanding Acting (Male) In the Eyes of a Killer', 'Hoboken International Film Festival 2008 Ganador Festival Award Best Actor In the Eyes of a Killer 2008 Nominado Festival Award Best Supporting Actor The Sensei', 'Atlantic City Cinefest 2009 Ganador Lifesaver Achievement Award']), ('Brooke Newton', 'NULL', ['Reparto', 'Producción', 'Sonido'], ['La casa de papel', 'Lupin', 'Ray Donovan'], 'NULL'), ('Marco Khan', 'NULL', ['Reparto', 'Especialistas', 'Producción'], 'NULL', 'NULL'), ('Leif Gantvoort', '1972', ['Reparto', 'Dirección', 'Guion'], ['Navy: Investigación criminal', 'The Amazing Spider-Man', 'Superpoli en Las Vegas'], ['Method Fest 2009 Nominado Best Actor Feature Film Lunatics, Lovers & Poets', 'Wisconsin Film Festival 2017 Ganador Jury Award Jury Award The 60 Yard Line Compartido con: Ryan Churchill · Nick Greco · Alex McCullough', 'Independent Filmmakers Showcase IFS Film Festival, US 2017 Ganador IFS Award Best Comedy Award The 60 Yard Line Compartido con: Ryan Churchill · Nick Greco · Alex McCullough', 'South Dakota Film Festival 2019 Ganador Festival Award Best Comedy Horror Heavy Stuff', 'Phoenix Comicon Film Festival 2017 Ganador Jury Prize Best of Festival The 60 Yard Line Compartido con: Ryan Churchill · Nick Greco · Alex McCullough']), ('Leonard Kelly-Young', '1948', ['Reparto', 'Reparto'], ['Perdida', 'Spanners', 'Justified: La ley de Raylan'], 'NULL'), ('Mary Christina Brown', 'NULL', ['Reparto', 'Producción', 'Reparto'], 'NULL', 'NULL')]
 16%|█▌        | 16/101 [2:20:29<11:30:16, 487.26s/it][('Alex Bulger', 'NULL', ['Cámara y electricidad', 'Reparto', 'Guion'], ['su papel en The Borrower', 'Muck', 'Sarcastic Actor'], 'NULL'), ('Jacob Schatz', '1991', ['Reparto', 'Dirección', 'Guion'], ['Three Headed Beast', 'Ultimate World', 'Autofiction'], 'NULL')]
 17%|█▋        | 17/101 [2:20:54<8:07:29, 348.21s/it] []
 18%|█▊        | 18/101 [2:32:10<10:18:00, 446.75s/it][('Rahul Madhav', '1986', ['Reparto', 'Reparto'], ['Thani Oruvan', 'Tiyaan', 'My Dear Machans'], 'NULL'), ('Vishnupriya', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Speed Track', 'Crime Story', 'V1 Murder Case'], 'NULL'), ('Anoop George', 'NULL', ['Reparto', 'Reparto'], ['su papel en Crime Story'], 'NULL'), ('Daniel Balaji', '1975', ['Reparto', 'Reparto'], ['Vettaiyaadu Vilaiyaadu', 'Maayavan', 'Vada Chennai'], 'NULL'), ('Kiran Raj', 'NULL', ['Reparto', 'Reparto'], ['su papel en Baba Kalyani', 'Son of Alibaba Nalpathonnaman', 'Hero'], 'NULL'), ('Thamburu', 'NULL', ['Reparto', 'Reparto'], ['su papel en Crime Story'], 'NULL'), ('Meenakshi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Crime Story'], 'NULL'), ('Babitha', 'NULL', ['Reparto', 'Reparto'], ['su papel en Crime Story', 'Hero', 'Padma'], 'NULL'), ('Shivaji Guruvayoor', 'NULL', ['Reparto', 'Reparto'], ['su papel en Driving Licence', 'Lucifer', 'Valimai'], 'NULL')]
 19%|█▉        | 19/101 [2:47:22<13:21:45, 586.65s/it][('Jay Baruchel', '1982', ['Reparto', 'Producción', 'Dirección'], ['Goon', 'Juerga hasta el fin', 'Cómo entrenar a tu dragón'], ['Annie Awards 2011 Ganador Annie Best Voice Acting in an Animated Feature Production Cómo entrenar a tu dragón For playing "Hiccup".', 'Canadian Comedy Awards 2010 Ganador Canadian Comedy Award Best Performance by a Male - Film The Trotsky', 'Daytime Emmy Awards 2019 Ganador Daytime Emmy Outstanding Performer in an Animated Program Dragones: Hacia nuevos confines For playing: "Hiccup".', 'Genie Awards 2011 Nominado Genie Best Performance by an Actor in a Leading Role The Trotsky', 'Jutra Awards 2011 Nominado Jutra Best Actor (Meilleur Acteur) The Trotsky']), ('Cate Blanchett', '1969', ['Reparto', 'Producción', 'Guion'], ['Carol', 'Blue Jasmine', 'El señor de los anillos: La comunidad del anillo'], ["Academy Awards, USA 2023 Nominado Oscar Best Performance by an Actress in a Leading Role Tár 2016 Nominado Oscar Best Performance by an Actress in a Leading Role Carol 2014 Ganador Oscar Best Performance by an Actress in a Leading Role Blue Jasmine 2008 Nominado Oscar Best Performance by an Actress in a Leading Role Elizabeth: La edad de oro 2008 Nominado Oscar Best Performance by an Actress in a Supporting Role I'm Not There 3 más", 'Academy of Science Fiction, Fantasy & Horror Films, USA 2022 Nominado Saturn Award Best Actress El callejón de las almas perdidas 2009 Nominado Saturn Award Best Actress El curioso caso de Benjamin Button 2007 Nominado Saturn Award Best Supporting Actress Diario de un escándalo 2004 Nominado Saturn Award Best Actress Desapariciones 2001 Nominado Saturn Award Best Actress Premonición', 'Film Critics Circle of Australia Awards 2024 Nominado FCCA Award Best Actress The New Boy 2024 Nominado FCCA Award Best Film The New Boy Compartido con: Lorenzo De Maio · Georgie Pym · Kath Shelper · Andrew Upton 2005 Ganador FCCA Award Best Actress in a Lead Role Little Fish 1999 Nominado FCCA Award Best Actor - Female Oscar y Lucinda 1998 Ganador FCCA Award Best Supporting Actor - Female Thank God He Met Lizzie', "Australian Film Institute 2008 Ganador AFI International Award Best Actress Elizabeth: La edad de oro 2005 Ganador AFI Award Best Lead Actress Little Fish 2005 Ganador News Limited Readers' Choice Award 1998 Nominado AFI Award Best Performance by an Actress in a Leading Role Oscar y Lucinda 1997 Ganador AFI Award Best Performance by an Actress in a Supporting Role Thank God He Met Lizzie", 'Blockbuster Entertainment Awards 2000 Nominado Blockbuster Entertainment Award Favorite Supporting Actress - Suspense El talento de Mr. Ripley']), ('Gerard Butler', '1969', ['Reparto', 'Producción', 'Cámara y electricidad'], ['300', 'Un ciudadano ejemplar', 'El fantasma de la ópera de Andrew Lloyd Webber'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2008 Nominado Saturn Award Best Actor 300', 'Annie Awards 2011 Nominado Annie Best Voice Acting in an Animated Feature Production Cómo entrenar a tu dragón For playing "Stoick".', 'Golden Camera, Germany 2016 Ganador Golden Camera Best International Actor For his body of work.', 'Satellite Awards 2005 Nominado Golden Satellite Award Best Actor in a Motion Picture, Comedy or Musical El fantasma de la ópera de Andrew Lloyd Webber', 'MTV Movie + TV Awards 2007 Nominado MTV Movie Award Best Performance 300 2007 Ganador MTV Movie Award Best Fight 300 Vs. the "Uber Immortal".']), ('Craig Ferguson', '1962', ['Guion', 'Música', 'Reparto'], ['Asuntos de famillia', 'Cómo entrenar a tu dragón 2', 'Winnie the Pooh'], ['British Independent Film Awards 2000 Nominado British Independent Film Award Best Screenplay El jardín de la alegría Compartido con: Mark Crowdy', 'Daytime Emmy Awards 2017 Nominado Daytime Emmy Outstanding Game Show Host Celebrity Name Game 2017 Nominado Daytime Emmy Outstanding Game Show Celebrity Name Game Compartido con: David Arquette · Courteney Cox · Richard Gerrits · Jennifer Mullin · John Quinn · Laura Robinson · · · 2016 Ganador Daytime Emmy Outstanding Game Show Host Celebrity Name Game 2015 Ganador Daytime Emmy Outstanding Game Show Host Celebrity Name Game', 'Primetime Emmy Awards 2006 Nominado Primetime Emmy Outstanding Individual Performance in a Variety or Music Program The Late Late Show with Craig Ferguson', 'Grammy Awards 2016 Nominado Grammy Best Comedy Album For the album "Just Being Honest." 2014 Nominado Grammy Best Comedy Album For the album "I\'m Here to Help." 2011 Nominado Grammy Best Spoken Word Album (Includes Poetry, Audio Books & Storytelling) For the album "American on Purpose"', "People's Choice Awards, USA 2015 Nominado People's Choice Award Favorite Late Night Talk Show Host"]), ('America Ferrera', '1984', ['Reparto', 'Producción', 'Dirección'], ['Barbie', 'Uno para todas', 'Cómo entrenar a tu dragón'], ['Academy Awards, USA 2024 Nominado Oscar Best Performance by an Actress in a Supporting Role Barbie', 'ALMA Awards 2014 Ganador ALMA Award Special Achievement in Film Cesar Chavez Compartido con: Diego Luna · Michael Peña · Rosario Dawson 2009 Nominado ALMA Award Actress in Television - Comedy Betty 2007 Ganador ALMA Award Outstanding Actress - Television Series, Mini-Series or Television Movie Betty 2006 Nominado ALMA Award Outstanding Actress in a Motion Picture Uno para todas', 'Critics Choice Awards 2024 Nominado Critics Choice Award Best Supporting Actress Barbie 2024 Ganador SeeHer Award', 'Primetime Emmy Awards 2008 Nominado Primetime Emmy Outstanding Lead Actress in a Comedy Series Betty For playing: "Betty Suarez". 2007 Ganador Primetime Emmy Outstanding Lead Actress in a Comedy Series Betty For playing: "Betty Suarez".', 'Golden Globes, USA 2009 Nominado Golden Globe Best Performance by an Actress in a Television Series - Comedy or Musical Betty 2008 Nominado Golden Globe Best Performance by an Actress in a Television Series - Comedy or Musical Betty 2007 Ganador Golden Globe Best Performance by an Actress in a Television Series - Comedy or Musical Betty']), ('Jonah Hill', '1983', ['Reparto', 'Producción', 'Guion'], ['Infiltrados en clase', 'Infiltrados en la universidad', 'El lobo de Wall Street'], ['Academy Awards, USA 2014 Nominado Oscar Best Performance by an Actor in a Supporting Role El lobo de Wall Street 2012 Nominado Oscar Best Performance by an Actor in a Supporting Role Moneyball: Rompiendo las reglas', 'American Comedy Awards, USA 2014 Nominado American Comedy Award Best Comedy Supporting Actor - Film Juerga hasta el fin', 'Art Film Festival 2019 Nominado Blue Angel Best Film En los 90 2019 Ganador Blue Angel Best Director En los 90', 'Berlin International Film Festival 2019 Nominado C.I.C.A.E. Award Panorama En los 90', 'BAFTA Awards 2012 Nominado BAFTA Film Award Best Supporting Actor Moneyball: Rompiendo las reglas']), ('Christopher Mintz-Plasse', '1989', ['Reparto', 'Producción', 'Guion'], ['Supersalidos', 'Kick Ass 2: Con un par', 'Kick-Ass: Listo para machacar'], ['MTV Movie + TV Awards 2008 Nominado MTV Movie Award Breakthrough Performance Supersalidos', "Teen Choice Awards 2014 Nominado Teen Choice Award Choice Movie: Fight Kick Ass 2: Con un par Dave Lizewski/Kick-Ass vs. Chris D'Amico/The Motherfucker Compartido con: Aaron Taylor-Johnson 2010 Nominado Teen Choice Award Choice Movie: Villain Kick-Ass: Listo para machacar", 'Young Artist Awards 2008 Nominado Young Artist Award Best Performance in a Feature Film - Supporting Young Actor Supersalidos', 'Young Hollywood Awards 2014 Nominado Young Hollywood Award Best Threesome Malditos vecinos Compartido con: Zac Efron · Dave Franco', 'Scream Awards 2010 Nominado Scream Award Best Supporting Actor Kick-Ass: Listo para machacar']), ('T.J. Miller', '1981', ['Reparto', 'Guion', 'Producción'], ['Silicon Valley', 'Monstruoso', 'Big Hero 6'], ['Scream Awards 2008 Nominado Scream Award Breakout Performance', "Critics Choice Television Awards 2016 Nominado Critics' Choice TV Award Best Supporting Actor in a Comedy Series Silicon Valley 2015 Ganador Critics' Choice TV Award Best Supporting Actor in a Comedy Series Silicon Valley", 'Behind the Voice Actors Awards 2016 Nominado BTVA Feature Film Voice Acting Award Best Vocal Ensemble in a Feature Film Hell and Back Compartido con: Nick Swardson · Bob Odenkirk · Mila Kunis · Rob Riggle · Susan Sarandon · Danny McBride · · · 2015 Nominado BTVA Feature Film Voice Acting Award Best Male Vocal Performance in a Feature Film in a Supporting Role Big Hero 6 As the voice of "Fred". 2015 Nominado BTVA Feature Film Voice Acting Award Best Vocal Ensemble in a Feature Film Big Hero 6 Compartido con: Ryan Potter · Scott Adsit · Daniel Henney · Jamie Chung · Genesis Rodriguez · Damon Wayans Jr. · · · 2015 Nominado BTVA Feature Film Voice Acting Award Best Vocal Ensemble in a Feature Film Cómo entrenar a tu dragón 2 Compartido con: Jay Baruchel · Cate Blanchett · Gerard Butler · Djimon Hounsou · America Ferrera · Kristen Wiig · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Kristen Wiig', '1973', ['Reparto', 'Producción', 'Guion'], ['La boda de mi mejor amiga', 'The Skeleton Twins', 'La vida secreta de Walter Mitty'], ['Academy Awards, USA 2012 Nominado Oscar Best Writing, Original Screenplay La boda de mi mejor amiga Compartido con: Annie Mumolo', 'American Comedy Awards, USA 2014 Nominado American Comedy Award Best Comedy Actress - Film La vida secreta de Walter Mitty 2014 Nominado American Comedy Award Best Comedy Supporting Actress - Film Los amos de la noticia', 'Annie Awards 2014 Nominado Annie Outstanding Achievement in Voice Acting in an Animated Feature Production Gru 2: Mi villano favorito As "Lucy"', 'BAFTA Awards 2012 Nominado BAFTA Film Award Best Original Screenplay La boda de mi mejor amiga Compartido con: Annie Mumolo', 'Critics Choice Awards 2015 Nominado Critics Choice Award Best Actress in a Comedy Movie The Skeleton Twins']), ('Djimon Hounsou', '1964', ['Reparto', 'Producción', 'Dirección'], ['Diamante de sangre', 'Guardianes de la Galaxia', 'En América'], ['Academy Awards, USA 2007 Nominado Oscar Best Performance by an Actor in a Supporting Role Diamante de sangre 2004 Nominado Oscar Best Actor in a Supporting Role En América', 'Acapulco Black Film Festival 2014 Nominado Hollywood Award Best Ensemble Cast Ligera de equipaje Compartido con: Adam Brody · Jill Scott · Derek Luke · Taye Diggs · Lauren London · Jenifer Lewis · Terrence Jenkins · · · · · · ·', 'Blockbuster Entertainment Awards 2001 Nominado Blockbuster Entertainment Award Favorite Supporting Actor - Action Gladiator (El gladiador)', 'Critics Choice Awards 2007 Nominado Critics Choice Award Best Supporting Actor Diamante de sangre', 'Chicago Film Critics Association Awards 1998 Nominado CFCA Award Most Promising Actor Amistad'])]
 20%|█▉        | 20/101 [3:00:52<14:42:30, 653.70s/it][('Ivan Sergei', '1971', ['Reparto', 'Dirección', 'Guion'], ['Lo opuesto al sexo', 'Jack & Jill', 'Crossing Jordan'], ['Best Actors Film Festival 2020 Ganador Best Actor Best Actor in a Comedy Magic Max', 'Culver City Film Festival 2020 Ganador Best Ensemble Cast Best Ensemble Cast - Feature Film Magic Max Compartido con: Parker Bates · Jud Tylor · Lotus Blossom · Barry Corbin · Sally Kirkland · Gary Hershberger · Joe Estevez · ·', 'Canadian International Faith & Family Film Festival 2020 Nominado CIFF Award Best Actor Magic Max', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Masiela Lusha', '1985', ['Reparto', 'Equipo ejecutivo', 'Producción'], ['El mal que esconde el olvido', 'El último Sharknado: Ya era hora', 'George Lopez'], 'NULL'), ('Drew Fuller', '1980', ['Reparto', 'Dirección', 'Producción'], ['El último regalo', 'Embrujadas', 'Army Wives'], 'NULL'), ('Krista Allen', 'NULL', ['Reparto', 'Banda sonora', 'Reparto'], ['Belleza y poder', 'Los vigilantes de la playa', 'Ejecutivo agresivo'], ['Daytime Emmy Awards 2023 Nominado Daytime Emmy Outstanding Performance by a Supporting Actress in a Daytime Drama Series Belleza y poder', 'Burbank International Film Festival 2018 Nominado Best Actress Best Actress Eleven Eleven', 'IndieFEST Film Awards 2018 Ganador Award of Merit Special Mention Leading Actress Eleven Eleven', 'Hollywood Boulevard Film Festival 2018 Ganador Festival Award Best Actress Eleven Eleven', 'Silver State Film Festival 2020 Ganador Festival Award Best supporting Actress - Feature Film Shadows']), ('Peter Dobson', '1964', ['Reparto', 'Producción', 'Dirección'], ['Forrest Gump', 'Agárrame esos fantasmas', '3 Days in Malay'], ["Accolade Competition 2016 Ganador Award of Excellence Special Mention Feature Film Bad Frank Compartido con: Tony Germinario · Kevin Interdonato · Brandon Heitkamp · Andy O'Connell · George Stohr · Joseph Sernio", "Action on Film International Film Festival, USA 2016 Nominado Action on Film Award Best Drama Feature Bad Frank Compartido con: Tony Germinario · Kevin Interdonato · George Stohr · Andy O'Connell · Brandon Heitkamp · Joseph Sernio", "Atlantic City Cinefest 2016 Ganador Festival Prize Best Feature - Action Drama Bad Frank Compartido con: Tony Germinario · Kevin Interdonato · Brandon Heitkamp · George Stohr · Andy O'Connell · Joseph Sernio 2012 Ganador Lifesaver Achievement Award", "New York City Independent Film Festival 2017 Nominado Best Feature Film Best Feature Film Bad Frank Compartido con: Tony Germinario · Kevin Interdonato · Brandon Heitkamp · George Stohr · Andy O'Connell · Russ Russo ·", "Maryland International Film Festival 2017 Ganador Best Feature Film Best Feature Film Bad Frank Compartido con: Brandon Heitkamp · Kevin Interdonato · Tony Germinario · Joseph Sernio · George Stohr · Andy O'Connell"]), ('Anne Winters', '1994', ['Reparto', 'Producción', 'Banda sonora'], ['Por trece razones', 'Escuela para fracasados', 'Zac and Mia'], ['Daytime Emmy Awards 2018 Ganador Daytime Emmy Outstanding Lead Actress in a Digital Daytime Drama Series Zac and Mia For playing: "Mia Phillips".', "Milano International Film Festival Awards (MIFF Awards) 2014 Ganador Leonardo's Horse Best Ensemble Cast Sand Castles Compartido con: Jordon Hodges · Randy Spence · Saxon Trainor · Daniella Grace · Scott Jemison · Clint Howard", 'St. Tropez International Film Festival 2016 Ganador Festival Prize Best Supporting Actress The Tribe', 'Christian Film Festival-Menchville Baptist Church 2016 Ganador Best Cast Pass the Light Compartido con: Cameron Palatas · Alexandria DeBerry · Dalpre Grayer · Jon Gries · Colby French · Milena Govich · · · · · · · · · · · · · · · · · · · · · · · 2016 Ganador Best Actress Teen Fan Favorite Pass the Light', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Adrian L. Tudor', 'NULL', ['Reparto', 'Reparto'], ['su papel en April Rain', 'Fatal instinct (Por instinto'], 'NULL'), ('Richard Burgi', '1958', ['Reparto', 'Reparto'], ['El infierno verde', 'Hostel 2', 'Mujeres desesperadas'], ['Screen Actors Guild Awards 2007 Nominado Actor Outstanding Performance by an Ensemble in a Comedy Series Mujeres desesperadas Compartido con: Andrea Bowen · Mehcad Brooks · Ricardo Chavira · Marcia Cross · James Denton · Teri Hatcher · · · · · · · · · · · · · · · · · ·', 'Accolade Competition 2015 Ganador Award of Excellence Supporting Actor No Letting Go', 'Sydney Indie Film Festival 2015 Nominado Festival Award Best Male Support No Letting Go', 'Gold Derby Awards 2006 Nominado Gold Derby TV Award Ensemble of the Year Mujeres desesperadas Compartido con: Andrea Bowen · Mehcad Brooks · Ricardo Chavira · Marcia Cross · James Denton · Harriet Sansom Harris · · · · · · · · · · · · · ·', 'Soap Hub Award 2021 Nominado 2nd Annual Soap Hub Award Favorite The Young & The Restless Actor The Young and the Restless']), ('Dominique Swain', '1980', ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], ['Lolita de Adrian Lyne', 'Cara a cara (Face/Off', 'Alpha Dog'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 1998 Nominado Saturn Award Best Performance by a Younger Actor/Actress Cara a cara (Face/Off)', 'Chicago Film Critics Association Awards 1999 Nominado CFCA Award Most Promising Actress Lolita de Adrian Lyne', 'MTV Movie + TV Awards 1999 Nominado MTV Movie Award Best Kiss Lolita de Adrian Lyne Compartido con: Jeremy Irons', 'Young Artist Awards 1999 Ganador Young Artist Award Best Performance in a TV Movie/Pilot/Mini-Series or Series - Leading Young Actress Lolita de Adrian Lyne', 'YoungStar Awards 1999 Nominado YoungStar Award Best Performance by a Young Actress in a Mini-Series/Made for TV Film Lolita de Adrian Lyne']), ('Scott Freeburg', 'NULL', ['Reparto', 'Reparto'], ['su papel en A dos metros bajo tierra', 'Entourage (El séquito', '¡Yo no he sido!'], 'NULL')]
 21%|██        | 21/101 [3:07:19<12:44:31, 573.39s/it][('Cody McGlashan', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Gary Screams for You', 'Another Day y See Me'], 'NULL'), ('Steffen Diem Garcia', 'NULL', ['Reparto', 'Equipos adicionales', 'Producción'], ['su papel en Rasta Powers', 'The Last House of Horror', 'Viral'], 'NULL'), ('Michael Stock', 'NULL', ['Reparto', 'Dirección de casting', 'Producción'], ['su papel en Gary Screams for You', 'Chicago Fire', "Uta Hagen's Acting Class"], 'NULL'), ('T.J. Thomas', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gary Screams for You'], 'NULL'), ('Rachel Blakes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gary Screams for You'], 'NULL')]
 22%|██▏       | 22/101 [3:20:00<13:49:20, 629.88s/it][('Martin Pechlát', '1974', ['Reparto', 'Banda sonora', 'Reparto'], ['Odpad mesto smrt', 'Okupace', 'Atlas ptáku'], 'NULL'), ('David Svehlík', '1972', ['Reparto', 'Guion y continuidad', 'Reparto'], ['Samhain, A la sombra', 'Ocas jesterky'], 'NULL'), ('Hynek Cermák', '1973', ['Reparto', 'Guion', 'Dirección'], ['Nevinnost', 'Stalingrado', 'Cirkus Bukowsky'], 'NULL'), ('Martin Hofmann', '1978', ['Reparto', 'Edición', 'Reparto'], ['Operación Anthropoid', 'El secreto de los hermanos Grimm', 'La casa de la esperanza'], 'NULL'), ('Daniela Kolárová', '1946', ['Reparto', 'Banda sonora', 'Reparto'], ['Kawasakiho ruze', 'Slasti Otce vlasti', 'Sueños de juventud'], 'NULL'), ('Zuzana Norisová', '1979', ['Reparto', 'Banda sonora', 'Reparto'], ['Rebelové', 'Tak fajn', 'Letiste'], 'NULL'), ('Jana Kolesárová', '1976', ['Reparto', 'Banda sonora', 'Reparto'], ['Plan de vuelo: Desaparecida', 'Blur', 'Bonds'], 'NULL'), ('Kristýna Boková', '1981', ['Reparto', 'Reparto'], ['Toman', 'Obcanský prukaz', 'Rapl'], 'NULL'), ('Monika Absolonová', '1976', ['Reparto', 'Banda sonora', 'Reparto'], ['Kleopatra', 'Vybíjená', 'Zdivocelá zeme'], 'NULL'), ('Lucie Cerná', '1995', ['Reparto', 'Reparto'], ['Nezné vlny', 'Los mosqueteros', 'Prvok, Sampon, Tecka a Karel'], 'NULL')]
 23%|██▎       | 23/101 [3:33:46<14:55:26, 688.80s/it][('Jonathan Bennett', '1981', ['Reparto', 'Producción', 'Dirección'], ['Chicas malas', 'La chica invisible', 'La casa de Navidad'], ['Palm Beach International Film Festival 2003 Ganador Best Actor Season of Youth', 'Teen Choice Awards 2004 Nominado Teen Choice Award Choice Movie Chemistry Chicas malas Compartido con: Lindsay Lohan 2004 Nominado Teen Choice Award Choice Breakout Movie Star - Male Chicas malas', 'Action on Film International Film Festival, USA 2013 Ganador AOF/WAB Award Outstanding Cast Performance Police Guys Compartido con: Jon Lee Brody · Stephen Kramer Glickman · Lester Speight · Shawna Waldron · Lisa Catara · · · · · · · · · · ·', 'Los Angeles International Underground Film Festival 2013 Nominado LAIUFF Award Best Actor Misogynist', 'North Hollywood Cinefest 2020 Nominado Festival Award Best Actor Take Me to Tarzana']), ('Talulah Riley', '1985', ['Reparto', 'Dirección', 'Guion'], ['Pride & Prejudice (Orgullo y prejuicio', 'Westworld', 'Origen'], ['Edinburgh International Film Festival 2015 Nominado Audience Award Scottish Mussel', 'Screen Actors Guild Awards 2017 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Westworld Compartido con: Ben Barnes · Ingrid Bolsø Berdal · Ed Harris · Luke Hemsworth · Anthony Hopkins · Sidse Babett Knudsen · · · · · · · · · ·', 'Garden State Film Festival 2019 Ganador Festival Award Best Supporting Actress - Feature Film El último testigo', 'Phoenix Film Critics Society Awards 2010 Nominado PFCS Award Best Ensemble Acting Origen Compartido con: Ken Watanabe · Leonardo DiCaprio · Joseph Gordon-Levitt · Tom Hardy · Elliot Page · Cillian Murphy · · · · · ·', 'Central Ohio Film Critics Association 2011 Nominado COFCA Award Best Ensemble Origen Compartido con: Leonardo DiCaprio · Joseph Gordon-Levitt · Tom Hardy · Elliot Page · Ken Watanabe · Cillian Murphy · · · · · ·']), ('Rosa Salazar', '1985', ['Reparto', 'Producción', 'Dirección'], ['Alita: Ángel de combate', 'El corredor del laberinto: Las pruebas', 'El corredor del laberinto: La cura mortal'], ['Imagen Foundation Awards 2019 Nominado Imagen Award Best Actress - Feature Film Alita: Ángel de combate', 'Sundance Film Festival 2017 Nominado Short Film Grand Jury Prize Good Crazy', 'Austin Film Critics Association 2020 Nominado AFCA Award Best Motion Capture/Special Effects Performance Alita: Ángel de combate', 'Washington DC Area Film Critics Association Awards 2019 Nominado WAFCA Award Best Motion Capture Performance Alita: Ángel de combate', 'Twin Cities Film Fest, US 2015 Ganador Breakthrough Achievement Award Performance Night Owls']), ('Samuel Hunt', '1986', ['Reparto', 'Dirección', 'Guion'], ['Submerged', 'Invencible (Unbroken - parte 2', 'La casa del terror (Haunt'], 'NULL'), ('Cody Christian', '1995', ['Reparto', 'Producción', 'Banda sonora'], ['Los sustitutos', 'Los muertos del hambre', 'Final Fantasy VII Remake'], 'NULL'), ('Giles Matthey', '1987', ['Reparto', 'Reparto'], ['Jobs', "Le Mans '66", '24: Vive otro día'], 'NULL'), ('Denzel Whitaker', '1990', ['Reparto', 'Dirección', 'Edición'], ['El gran debate', 'Black Panther', 'Training Day (Día de entrenamiento'], 'NULL'), ('Willa Ford', '1981', ['Reparto', 'Composición', 'Guion'], ['Viernes 13', 'Universal Squadrons', 'Undercover brother (El hermano secreto'], 'NULL'), ('Mario Van Peebles', '1957', ['Reparto', 'Dirección', 'Producción'], ["How to Get the Man's Foot Outta Your Ass", 'El sargento de hierro', 'Tiburón, la venganza'], ['Directors Guild of America, USA 1991 Nominado DGA Award Outstanding Directorial Achievement in Dramatic Shows - Daytime CBS Schoolbreak Special For episode "Malcolm Takes a Shot".', "Gijón International Film Festival 2004 Nominado Grand Prix Asturias Best Feature How to Get the Man's Foot Outta Your Ass", 'Image Awards (NAACP) 2022 Nominado Image Award Outstanding Directing in a Television Movie or Special Salt-N-Pepa 2017 Nominado Image Award Outstanding Directing in a Motion Picture (Television) Raíces Episode: "Part 2 (2016)" 2013 Nominado Image Award Outstanding Directing in a Drama Series Boss For episode "Backflash" 2005 Nominado Image Award Outstanding Actor in a Motion Picture How to Get the Man\'s Foot Outta Your Ass 2002 Nominado Image Award Outstanding Supporting Actor in a Motion Picture Ali 1 más', "Film Independent Spirit Awards 2005 Nominado Independent Spirit Award Best Feature How to Get the Man's Foot Outta Your Ass 2005 Nominado Independent Spirit Award Best Director How to Get the Man's Foot Outta Your Ass 2005 Nominado Independent Spirit Award Best Screenplay How to Get the Man's Foot Outta Your Ass Compartido con: Dennis Haggerty", 'Locarno International Film Festival 1995 Ganador Silver Leopard Panther 1995 Ganador Prize of the Ecumenical Jury - Special Mention Panther 1995 Nominado Golden Leopard Panther']), ('Tim Daly', '1956', ['Reparto', 'Producción', 'Dirección'], ['Basic', 'El fugitivo'], ['Daytime Emmy Awards 2006 Nominado Daytime Emmy Outstanding Performer in a Children/Youth/Family Special Edge of America For playing: "Leroy McKinney".', 'Primetime Emmy Awards 2007 Nominado Primetime Emmy Outstanding Guest Actor in a Drama Series Los Soprano For playing "J.T. Dolan".', 'Satellite Awards 2001 Ganador Golden Satellite Award Best Performance by an Actor in a Series, Drama El fugitivo. La caza continúa', 'Razzie Awards 1996 Nominado Razzie Award Worst Screen Couple Dr. Jekyll y Miss. Hyde Compartido con: Sean Young', 'Screen Actors Guild Awards 2001 Nominado Actor Outstanding Performance by a Male Actor in a Drama Series El fugitivo. La caza continúa'])]
 24%|██▍       | 24/101 [3:40:18<12:49:29, 599.61s/it][('Vincent Ticali', '1960', ['Reparto', 'Reparto'], ['All the Little Things We Kill', 'The Blacklist', 'Blindspot'], ['Accolade Competition 2024 Ganador Award of Merit Special Mention Short Film Through the Modern Looking Glass Compartido con: Christina Renee · Jam Murphy · Polly Kreisman · Joaquín Pittaluga · Diletta Guglielmi · Christopher Alexander Chukwueke · · · · · · · ·', "Manhattan Film Festival 2023 Ganador Best Cast - Short Best Cast Aaron with 2 A's Compartido con: Steve Lichtenstein · Pamela Jayne Morgan · Anthony Robert Grasso · Mark Resnik · Devi Morgan · · · ·", 'Best Shorts Competition 2024 Ganador Award of Excellence Ensemble Cast Through the Modern Looking Glass Compartido con: Christina Renee · Jam Murphy · Luciana VanDette · Daniel Rios Jr. · Polly Kreisman · Julianna Zinkel · · · ·', 'Chelsea Film Festival 2017 Nominado Festival Award Best Short Film Tu Me Manque Compartido con: Stefano Da Fre · Laura Pellegrini · Brandon Delosreyes · Dejun Wong · Krysia Carter-Giez · Iorgo Papoutsas · · · · ·', 'Silk Road International Film Festival Ireland 2018 Ganador Festival Prize Best International Short Film Tu Me Manque Compartido con: Stefano Da Fre · Laura Pellegrini · Brandon Delosreyes · Dejun Wong · Krysia Carter-Giez · Iorgo Papoutsas · · · · ·']), ('Aoife Williamson', 'NULL', ['Reparto', 'Producción', 'Equipos adicionales'], ['su papel en Work', 'The Single Man', 'Proslogion'], 'NULL'), ('Jeffrey Fuller', 'NULL', ['Reparto', 'Reparto'], ['su papel en Deviant in Blue, Ley y orden', 'Work'], 'NULL'), ('Krystal Renée', 'NULL', ['Reparto', 'Reparto'], ['su papel en Julie entre bambalinas', 'Work'], 'NULL'), ('DMac Burns', 'NULL', ['Reparto', 'Composición', 'Reparto'], ['su papel en Work'], 'NULL')]
 25%|██▍       | 25/101 [3:51:28<13:06:09, 620.65s/it][('Sung Ho Park', 'NULL', ['Reparto', 'Dirección', 'Reparto'], ['su papel en Uncharted: Riches of Tyranny (Fan Film', 'Gun Mountain y Me and the Boys'], 'NULL'), ('Tony Winkler', 'NULL', ['Reparto', 'Reparto'], ['su papel en Me and the Boys'], 'NULL'), ('Roger Montero', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gun Mountain y Me and the Boys'], 'NULL'), ('Andrew Siaterlis', 'NULL', ['Reparto', 'Dirección', 'Guion'], ["su papel en Me and the Boys, Gun Mountain y Uncharted 2: Leif's Deception (Fan Film"], 'NULL'), ('Skyler Leming', 'NULL', ['Reparto', 'Reparto'], ["su papel en Me and the Boys, Uncharted 2: Leif's Deception (Fan Film", "Uncharted 3: Drake's Legacy (Fan Film"], 'NULL'), ('Nathan Edwards', 'NULL', ['Reparto', 'Reparto'], ['su papel en Me and the Boys'], 'NULL'), ('Joseph Watson', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Me and the Boys'], 'NULL'), ('Shane Hansen', 'NULL', ['Reparto', 'Guion', 'Dirección'], ["su papel en Uncharted 2: Leif's Deception (Fan Film", 'Uncharted: Riches of Tyranny (Fan Film', 'Me and the Boys'], 'NULL'), ('Jayson Geveshausen', 'NULL', ['Reparto', 'Reparto'], ['su papel en Me and the Boys'], 'NULL')]
 26%|██▌       | 26/101 [4:03:10<13:26:15, 645.00s/it][('Aimee Theresa', 'NULL', ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], 'NULL', ['Media Film Festival, US 2024 Ganador Jury Award Golden Trolley Award for Outstanding Achievement The Woman in the Room Compartido con: Nicholas T. Peetros · Danny Donnelly · Eric Crist · Susan Moses · Brian Anthony Wilson', 'IndieFEST Film Awards 2021 Ganador Award of Excellence Best Film The Arrangement Compartido con: Jake Hunsicker · Andrew Hunsicker · Melissa Hunsicker · Danny Donnelly · Jennifer M. Kay', 'Independent Horror Movie Awards 2023 Ganador Winter Award Best Writing The Woman in the Room 2023 Nominado Winter Award Best Short The Woman in the Room Compartido con: Nicholas T. Peetros · Danny Donnelly', 'Paris Play Film Festival 2021 Nominado Festival Prize Best Feature Film The Arrangement Compartido con: Jake Hunsicker · Andrew Hunsicker · Melissa Hunsicker · Danny Donnelly · Jennifer M. Kay', 'The Indie Gathering International Film Festival. 2020 Nominado Festival Prize Best Supporting Actress Viscera']), ('Jennifer M. Kay', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Last Known Location', 'Certifiable', 'The Arrangement'], ['IndieFEST Film Awards 2021 Ganador Award of Excellence Best Film The Arrangement Compartido con: Jake Hunsicker · Andrew Hunsicker · Melissa Hunsicker · Danny Donnelly · Aimee Theresa', 'Paris Play Film Festival 2021 Nominado Festival Prize Best Feature Film The Arrangement Compartido con: Jake Hunsicker · Andrew Hunsicker · Melissa Hunsicker · Danny Donnelly · Aimee Theresa', 'Golden Lemur International Film Festival 2022 Nominado May Award Best Producer The Arrangement Compartido con: Andrew Hunsicker · Jake Hunsicker · Melissa Hunsicker · Danny Donnelly · Aimee Theresa', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Annie Raczko', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Battle', 'Road Rage', 'Cheat-hos: A Political Comedy'], 'NULL'), ('Andrew Hunsicker', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en House of Cards', 'The Arrangement', 'Habemus Papam: Una historia de poder'], ['Best Shorts Competition 2023 Ganador Award of Excellence Ensemble Cast Meatloaf and Pie at the Sunset Cafe Compartido con: Dax Richardson · Sophia Lucia Parola · Glen Macnow · Jake Hunsicker', 'IndieFEST Film Awards 2024 Ganador Award of Merit Special Mention Best Ensemble Cast Meatloaf and Pie at the Sunset Cafe Compartido con: Dax Richardson · Sophia Lucia Parola · Jake Hunsicker · Glen Macnow 2021 Ganador Award of Excellence Best Film The Arrangement Compartido con: Jake Hunsicker · Melissa Hunsicker · Danny Donnelly · Aimee Theresa · Jennifer M. Kay', 'Paris Play Film Festival 2021 Nominado Festival Prize Best Feature Film The Arrangement Compartido con: Jake Hunsicker · Melissa Hunsicker · Danny Donnelly · Aimee Theresa · Jennifer M. Kay', 'Golden Lemur International Film Festival 2022 Nominado May Award Best Producer The Arrangement Compartido con: Jake Hunsicker · Melissa Hunsicker · Jennifer M. Kay · Danny Donnelly · Aimee Theresa', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Frank Williams', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en El fenómeno Cazafantasmas', 'I Am Santa Claus', 'Mancation'], 'NULL'), ('Melissa Hunsicker', 'NULL', ['Producción', 'Reparto', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en The Arrangement', 'Dead by Noon', 'Post Hit Measures'], ['IndieFEST Film Awards 2021 Ganador Award of Excellence Best Film The Arrangement Compartido con: Jake Hunsicker · Andrew Hunsicker · Danny Donnelly · Aimee Theresa · Jennifer M. Kay', 'Paris Play Film Festival 2021 Nominado Festival Prize Best Feature Film The Arrangement Compartido con: Jake Hunsicker · Andrew Hunsicker · Danny Donnelly · Aimee Theresa · Jennifer M. Kay', 'Golden Lemur International Film Festival 2022 Nominado May Award Best Producer The Arrangement Compartido con: Andrew Hunsicker · Jake Hunsicker · Jennifer M. Kay · Danny Donnelly · Aimee Theresa', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Cornelius Bass', 'NULL', ['Reparto', 'Reparto'], ['su papel en Something Borrowed'], 'NULL'), ('Matthew Brady', 'NULL', ['Reparto', 'Reparto'], ['su papel en Something Borrowed'], 'NULL'), ('Gregory Scott Campbell', 'NULL', ['Reparto', 'Reparto'], ['su papel en Something Borrowed'], 'NULL')]
 27%|██▋       | 27/101 [4:14:22<13:25:40, 653.26s/it][('Ste Bergin', '1997', ['Reparto', 'Producción', 'Dirección'], ['A Manchester Story', 'Anne Droid y Revolution C'], 'NULL'), ('Pete Garvey', 'NULL', ['Reparto', 'Casting', 'Equipos adicionales'], ['su papel en Ammonium Nitrate', 'Tomorrow at Noon', 'Zombie Cats on Mars: Life on Mars Parody'], 'NULL'), ('Elle Finch', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ammonium Nitrate'], 'NULL'), ('Sean McLaughlin', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ammonium Nitrate'], 'NULL'), ('Jake Thomas Owens', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ammonium Nitrate', 'Levelling the Score'], 'NULL'), ('James Gleave', 'NULL', ['Dirección', 'Guion', 'Edición'], ['su papel en Ammonium Nitrate', 'DwP&Me'], 'NULL'), ('Drew Jenkinson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ammonium Nitrate', 'Hollyoaks', 'Levelling the Score'], 'NULL'), ('Michael L. Knight', 'NULL', ['Reparto', 'Reparto'], ['su papel en Corruption II', 'Ammonium Nitrate'], 'NULL'), ('Adam Ahmed', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ammonium Nitrate', 'Guilty Pleasures (POC'], 'NULL')]
 28%|██▊       | 28/101 [4:26:26<13:40:31, 674.40s/it][('Vida Guerra', '1974', ['Reparto', 'Reparto'], ['CHiPs: Loca patrulla motorizada', 'Mercy for Angels', 'Scarface: The World Is Yours'], 'NULL'), ('Emilio Rivera', '1961', ['Reparto', 'Equipos adicionales', 'Producción'], ['Hijos de la anarquía', 'Mayans M'], ['Imagen Foundation Awards 2023 Nominado Imagen Award Best Supporting Actor - Drama (Television) Mayans M.C.', 'SoCal Independent Film Festival 2014 Nominado SoCal IFF Award Best Actor Sins of the Father 2014 Nominado Best Actor Sins of the Father', 'DOC LA 2018 Ganador DOC LA Award DOC LA Storyteller Award Soul of Lincoln Heights', 'Vancouver Badass Film Festival 2018 Nominado Badass Best Actor Badsville', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('John Amos', '1939', ['Reparto', 'Guion', 'Producción'], ['El príncipe de Zamunda', 'Encerrado', 'Good Times'], ['CableACE Awards 1993 Nominado CableACE Actor in a Dramatic Series Without a Pass Televised as "Showtime 30-Minute Movie: Without a Pass"', 'Primetime Emmy Awards 1977 Nominado Primetime Emmy Outstanding Lead Actor for a Single Appearance in a Drama or Comedy Series Raíces For playing "Toby". For episode: "Part V".', 'Image Awards (NAACP) 1996 Nominado Image Award Outstanding Supporting Actor in a Comedy Series In the House', "TV Land Awards 2007 Ganador Anniversary Award Raíces For its 30th anniversary. Compartido con: LeVar Burton · Louis Gossett Jr. · Leslie Uggams · Ben Vereen · Todd Bridges · Cicely Tyson · · · 2006 Ganador Impact Award Good Times Compartido con: Ralph Carter · BernNadette Stanis · Jimmie 'JJ' Walker · Ja'net DuBois 2004 Ganador Groundbreaking Show La chica de la tele Compartido con: Mary Tyler Moore · Edward Asner · Valerie Harper · Cloris Leachman · Gavin MacLeod · Betty White", 'DVD Exclusive Awards 2005 Nominado DVDX Award Best Actor (in a DVD Premiere Movie) The Watermelon Heist']), ('Tai Carlton', 'NULL', ['Reparto', 'Música', 'Reparto'], ['su papel en Mercy for Angels', 'South Park', 'Syphon Gun'], 'NULL'), ('Roberto Sanchez', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['2 Fast 2 Furious: A todo gas 2', 'Black Mafia Family', 'Palm Royale'], ['Imagen Foundation Awards 2019 Nominado Imagen Award Best Short Film The Terms of Us 2015 Nominado Imagen Award Best Actor - Feature Film Lake Los Angeles', 'Santa Clarita International Film Festival 2022 Ganador Acting Award Best Actor Testimony', 'Santa Fe Film Festival 2020 Ganador Best in Latin Cine Best Short Film The Terms of Us', 'WorldFest Houston 2020 Ganador Platinum Remi Award Best Short Film- Dramatic Original The Terms of Us', 'Accolade Competition 2015 Ganador Award of Merit Latino / Hispanic Lake Los Angeles Compartido con: Mike Ott · Atsuko Okatsuka 2015 Ganador Award of Merit Lead Actor Lake Los Angeles']), ('Lawrence Hilton-Jacobs', '1953', ['Reparto', 'Producción', 'Dirección'], ['Los Jackson', 'L'], ['CableACE Awards 1995 Nominado CableACE Performance in a Comedy Special 1995 MTV Movie Awards Compartido con: Danny Bonaduce · Franklin Cover · Frank Gorshin · Robert Hegyes · Sherman Hemsley · Shirley Jones · · · ·', 'Daytime Emmy Awards 2021 Nominado Daytime Emmy Outstanding Lead Actor in a Daytime Fiction Program A House Divided For playing: "Cameran Sanders".', 'Image Awards (NAACP) 1993 Nominado Image Award Outstanding Lead Actor in a Drama Series, Mini-Series or Television Movie Los Jackson', 'TV Land Awards 2007 Ganador Anniversary Award Raíces For its 30th anniversary. Compartido con: LeVar Burton · Louis Gossett Jr. · Leslie Uggams · John Amos · Ben Vereen · Todd Bridges · Cicely Tyson · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Nicola Bertram', 'NULL', ['Reparto', 'Reparto'], ['su papel en Para toda la humanidad', 'Bosch: Legacy', 'Hollywood'], 'NULL'), ('Sofie Norman', 'NULL', ['Reparto', 'Reparto'], 'NULL', 'NULL'), ('Angela Pinner', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mercy for Angels', 'Detained in the Desert', 'Hauntsville'], 'NULL')]
 29%|██▊       | 29/101 [4:32:53<11:46:00, 588.34s/it][('Maya Robinson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Nowhere Boys', 'Bad Mothers', 'Nowhere Boys: The Book of Shadows'], 'NULL'), ('Scott Johnson', 'NULL', ['Reparto', 'Casting', 'Reparto'], ['su papel en El especialista', 'Blue Heelers', 'Vecinos'], 'NULL'), ('Melanie Zanetti', '1985', ['Reparto', 'Reparto'], ['Bluey', 'Head Count', 'De amor y monstruos'], 'NULL'), ('Scarlett Dorrington', 'NULL', ['Reparto', 'Reparto'], ['su papel en Get Krack!n', 'The Sky Is Not Mine y Alba'], 'NULL'), ('Nathalie Santillana', 'NULL', ['Reparto', 'Reparto'], ['su papel en Alba'], 'NULL')]
 30%|██▉       | 30/101 [4:33:18<8:16:00, 419.17s/it] []
 31%|███       | 31/101 [4:36:10<6:42:38, 345.12s/it][('Shiva Ayama', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Checking Out, The Masterpiece', 'Charlie'], 'NULL'), ('Amir Malaklou', '1989', ['Reparto', 'Producción', 'Guion'], ['The Endgame', 'Evil', 'Viper Club'], 'NULL')]
 32%|███▏      | 32/101 [4:49:12<9:07:31, 476.10s/it][('Crina Semciuc', '1986', ['Reparto', 'Reparto'], ['Oameni de treaba', 'Wilde Maus', 'Selfie'], 'NULL'), ('Diana Gursca', 'NULL', ['Reparto', 'Reparto'], ['su papel en WebSiteStory'], 'NULL'), ('Oreste Alexandru Scarlat Teodorescu', 'NULL', ['Reparto', 'Reparto'], ['su papel en WebSiteStory', 'Eminescu versus Eminem', 'Si caii sunt verzi pe pereti'], 'NULL'), ('Constantin Bojog', 'NULL', ['Reparto', 'Reparto'], ['su papel en 4 meses, 3 semanas, 2 días', 'WebSiteStory', 'Bucuresti NonStop'], 'NULL'), ('Lia Bugnar', '1969', ['Reparto', 'Guion', 'Reparto'], ['Octav', 'Hello! How Are You?', 'La marca del lobo'], ['Accolade Competition 2018 Ganador Award of Merit Special Mention Supporting Actress Octav', 'Festigious International Film Festival 2018 Ganador Festival Award Best Actress Octav', 'Los Angeles Film Awards 2018 Ganador Jury Award Best Supporting Actress Octav', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Florin Piersic Jr.', '1968', ['Reparto', 'Guion', 'Dirección'], ['Killing Time', 'La protegida', 'Calatoria lui Gruber'], ['Warsaw International Film Festival 2012 Nominado Free Spirit Award Killing Time 2010 Nominado Best Short Killing Time', 'Transilvania International Film Festival 2022 Nominado Romanian Days Award Best Feature Film [Nothing] about Love', 'Gopo Awards, Romania 2015 Ganador Gopo Award Best Actor in a Leading Role Quod erat demonstrandum 2010 Nominado Gopo Award Best Actor in a Leading Role Calatoria lui Gruber', 'NexT International Film Festival 2009 Nominado Festival Award Best script The Drug Dealer 2009 Nominado Festival Award Best Romanian Short The Drug Dealer Compartido con: Vlad Feier · Razvan Marinescu', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Dragos Bucur', '1977', ['Reparto', 'Producción', 'Agente de talentos'], ['Camino a la libertad', 'Vaya patrulla', 'Hawaii'], ['Berlin International Film Festival 2010 Ganador EFP Shooting Star Romania.', 'Buenos Aires International Festival of Independent Cinema 2010 Ganador Best Actor Politist, adjectiv', 'Romanian Union of Filmmakers 2008 Ganador Romanian Union of Filmmakers Award Best Actor (Premiul pentru interpretare rol principal masculin) Boogie 2003 Ganador Romanian Union of Filmmakers Award Best Actor (Premiul pentru interpretare rol principal masculin) Furia Tied with Alexandru Papadopol for Occident (2002).', 'Tiburon International Film Festival 2019 Ganador Golden Reel Award Best Actor Hawaii', 'Gopo Awards, Romania 2024 Nominado Gopo Award Best Feature Film (Cel mai bun film de lung metraj) MMXX Compartido con: Dorian Boguta · Cristi Puiu · Anca Puiu 2021 Nominado Gopo Award Best Actor in a Supporting Role (Cel mai bun actor într-un rol secundar) Urma 2017 Nominado Gopo Award Best Feature Film (Cel mai bun film de lung metraj) Doua lozuri 2010 Ganador Gopo Award Best Actor in a Leading Role Politist, adjectiv 2009 Ganador Gopo Award Best Actor in a Leading Role Boogie']), ('Serban Ionescu', '1950', ['Reparto', 'Reparto'], ['Maria', 'Padureanca', 'Imposibila iubire'], 'NULL'), ('Cosmin Nedelcu', 'NULL', ['Producción', 'Reparto', 'Guion'], ['su papel en Teambuilding', 'Klaus & Barroso', '5Gang: Un Altfel de Craciun'], 'NULL'), ('Martin Stanciu', 'NULL', ['Reparto', 'Reparto'], ['su papel en WebSiteStory'], 'NULL')]
 33%|███▎      | 33/101 [4:54:32<8:06:43, 429.46s/it][('Ailish Symons', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en The Widow', 'Coyote', 'Capture'], 'NULL'), ('Jason Kavan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Emmerdale', 'Nice Mike y The Napoleon Complex'], 'NULL'), ('Alice Baxter', 'NULL', ['Reparto', 'Reparto'], ['su papel en Journey', 'World Business Report'], 'NULL'), ('Javan Hirst', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rey Arturo: La leyenda de Excalibur', 'Nina Forever', 'The Return'], 'NULL')]
 34%|███▎      | 34/101 [5:07:13<9:50:27, 528.78s/it][('Margarita Abroskina', '1994', ['Reparto', 'Reparto'], ['Dokumentalist'], 'NULL'), ('Mikhail Bashkatov', '1981', ['Producción', 'Reparto', 'Guion'], ['14+', 'Uchilki v zakone', 'Lyubimtsy'], 'NULL'), ('Oleg Chugunov', '2004', ['Reparto', 'Reparto'], ['The Blackout: La invasión', 'Igor Grom contra el Doctor Peste', 'Bender'], 'NULL'), ('Roman Chumakov', '1984', ['Dirección', 'Reparto', 'Guion'], ['Beef'], 'NULL'), ('Aleksey Dmitriev', '1976', ['Reparto', 'Reparto'], ['Zapreshchyonnaya realnost', 'La isla habitada', 'La fortaleza Brest'], 'NULL'), ('Fyodor Dobronravov', '1961', ['Reparto', 'Producción', 'Reparto'], ['Svaty', 'Ot pechali do radosti', 'Zhili-byli'], 'NULL'), ('Ulyana Donskova', 'NULL', ['Reparto', 'Reparto'], ['su papel en Byt', 'Selfi#Selfie', 'Istorik'], 'NULL'), ('Oleg Gaas', '1994', ['Reparto', 'Reparto'], ['Ten'], 'NULL'), ('Dmitriy Gogu', '1996', ['Reparto', 'Reparto'], ['Byt', 'Problemas con suerte', 'Poteryannye'], 'NULL'), ('Yuriy Kasmynin', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en Sin amor', 'Byt', 'Hacia el lago'], 'NULL')]
 35%|███▍      | 35/101 [5:11:27<8:10:51, 446.23s/it][('Sandy Jack', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Dead City', 'A Life More Ordinary', 'Dead City 2'], 'NULL'), ('Rohan Murphy', 'NULL', ['Sonido', 'Reparto', 'Composición'], ['su papel en Dead City', 'Dead City 2', 'The Bad Superhero'], 'NULL'), ('Stephen Samson', '1982', ['Reparto', 'Dirección', 'Guion'], ['Aquaman y el reino perdido', 'Time Spent', 'Preston Watson the Flying Scotsman'], ['Festival of Fantastic Films, UK 2023 Nominado Best Independent Short Award Best Short Film Time Spent Compartido con: Louis Blain Russell · Eleanor Griffiths · Tiffany Boysell · Danika Fenty · Gwyneth Evans · Rob Hutt', 'Miami Film Festival 2013 Nominado Short Film Award Best Short Film The Bad Superhero Compartido con: Sandy Jack', 'Around International Film Festival 2023 Ganador ARFF Amsterdam Audience Award Time Spent Compartido con: Rob Hutt · Celia Dumas · Louis Blain Russell · Eleanor Griffiths · Tiffany Boysell · Danika Fenty · · 2023 Nominado ARFF Amsterdam BEST ACTRESS & ACTORS AWARD Time Spent Compartido con: Louis Blain Russell', 'South Film and Arts Academy Festival 2023 Ganador Short Film Cast and Crew Award Best Supporting Actor Time Spent', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir'])]
 36%|███▌      | 36/101 [5:18:13<7:50:30, 434.31s/it][('Stephanie Belding', 'NULL', ['Reparto', 'Cámara y electricidad', 'Banda sonora'], ['Watchmen', 'eXistenZ', 'The void (El vacío'], 'NULL'), ('Steven McCarthy', 'NULL', ['Reparto', 'Guion', 'Producción'], 'NULL', 'NULL'), ('Harmeet Bhatti', 'NULL', ['Reparto', 'Guion', 'Reparto'], ["su papel en Kim's Convenience", 'Madres trabajadoras', "Everyone's Leaving Us"], 'NULL'), ('Tamara Almeida', 'NULL', ['Reparto', 'Producción', 'Equipos adicionales'], ['su papel en El escritor fantasma', 'Jane', 'What We Do for Love'], 'NULL'), ('Alyx Melone', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Talking Heads', 'Blood Child', 'O Negative'], 'NULL')]
 37%|███▋      | 37/101 [5:24:11<7:18:46, 411.36s/it][('Naomi Watts', '1968', ['Reparto', 'Producción', 'Banda sonora'], ['Mulholland Drive', 'King Kong', 'Birdman o (La inesperada virtud de la ignorancia'], ['Academy Awards, USA 2013 Nominado Oscar Best Performance by an Actress in a Leading Role Lo imposible 2004 Nominado Oscar Best Actress in a Leading Role 21 gramos', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2013 Nominado Saturn Award Best Actress Lo imposible 2008 Nominado Saturn Award Best Actress Promesas del este 2006 Ganador Saturn Award Best Actress King Kong 2003 Ganador Saturn Award Best Actress The Ring (La señal) 2002 Nominado Saturn Award Best Actress Mulholland Drive', 'Film Critics Circle of Australia Awards 2022 Nominado FCCA Award Best Actress La familia Bloom 2014 Ganador FCCA Award Best Actress Dos madres perfectas', 'Australian Film Institute 2010 Nominado AFI International Award Best Actress Madres & hijas 2006 Nominado AFI International Award Best Actress King Kong 2004 Ganador Global Achievement Award', 'Bambi Awards 2019 Ganador Bambi Film - International']), ('Billy Howle', '1989', ['Reparto', 'Reparto'], ['El rey proscrito', 'Star Wars: El ascenso de Skywalker', 'Dunkerque'], 'NULL'), ("Denis O'Hare", '1962', ['Reparto', 'Guion', 'Producción'], ['Dallas Buyers Club', 'La proposición', 'Algo en común'], ['Critics Choice Awards 2009 Ganador Critics Choice Award Best Acting Ensemble Mi nombre es Harvey Milk Compartido con: Sean Penn · James Franco · Josh Brolin · Victor Garber · Stephen Spinella · Emile Hirsch · Diego Luna', 'Primetime Emmy Awards 2017 Nominado Primetime Emmy Outstanding Guest Actor in a Drama Series This Is Us For playing: "Jessie". For episode: "Last Christmas". 2015 Nominado Primetime Emmy Outstanding Supporting Actor in a Limited Series or a Movie American Horror Story For playing: "Stanley". 2012 Nominado Primetime Emmy Outstanding Supporting Actor in a Miniseries or a Movie American Horror Story For playing "Larry Harvey".', 'Satellite Awards 2011 Ganador Special Achievement Award Best Ensemble, Television American Horror Story Compartido con: Connie Britton · Dylan McDermott · Evan Peters · Taissa Farmiga · Jessica Lange · Frances Conroy', 'Screen Actors Guild Awards 2020 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Big Little Lies Compartido con: Iain Armitage · Darby Camp · Chloe Coleman · Cameron Crovetti · Nicholas Crovetti · Laura Dern · · · · · · · · · · · · · · · · 2014 Nominado Actor Outstanding Performance by a Cast in a Motion Picture Dallas Buyers Club Compartido con: Jennifer Garner · Matthew McConaughey · Jared Leto · Dallas Roberts · Steve Zahn 2009 Nominado Actor Outstanding Performance by a Cast in a Motion Picture Mi nombre es Harvey Milk Compartido con: Josh Brolin · Joseph Cross · James Franco · Victor Garber · Emile Hirsch · Diego Luna · Sean Penn ·', 'Online Film & Television Association 2015 Nominado OFTA Television Award Best Supporting Actor in a Motion Picture or Miniseries American Horror Story For the fourth season entitled "Freak Show".']), ('Parker Sawyers', '1983', 'NULL', ['Michelle & Obama', 'Caza mayor', 'La momia'], ['', '', '', '', ''])]
 38%|███▊      | 38/101 [5:24:35<5:10:04, 295.31s/it][]
 39%|███▊      | 39/101 [5:32:06<5:53:21, 341.97s/it][('Audrey Rocard', 'NULL', ['Reparto', 'Reparto'], ['su papel en À Propos du Code'], 'NULL'), ('Joris Masson', 'NULL', ['Reparto', 'Reparto'], ['su papel en À Propos du Code'], 'NULL'), ('Claude Madelon', 'NULL', ['Reparto', 'Reparto'], ['su papel en À Propos du Code'], 'NULL'), ('Jean-Claude Eygreteau', 'NULL', ['Reparto', 'Reparto'], ['su papel en À Propos du Code'], 'NULL'), ('Yannick Moreau', 'NULL', ['Reparto', 'Reparto'], ['su papel en À Propos du Code'], 'NULL'), ('Blair Gannon', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en À Propos du Code', 'Space on the Corner', 'Just Talk'], 'NULL')]
 40%|███▉      | 40/101 [5:36:13<5:18:34, 313.36s/it][('Jordan Lawson', '1988', ['Reparto', 'Producción', 'Guion'], ['Nostalgic', 'Next Level', 'The Black Prison'], 'NULL'), ('Joanna Leonard', 'NULL', ['Reparto', 'Reparto'], ['su papel en Overdue Newsies', 'DeTour', 'The Valentine Nightmare'], 'NULL'), ('Shri Prasham', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en The Valentine Nightmare', 'Roommates', 'Overdue Newsies'], 'NULL')]
 41%|████      | 41/101 [5:37:29<4:02:05, 242.08s/it][]
 42%|████▏     | 42/101 [5:38:11<2:59:12, 182.25s/it][]
 43%|████▎     | 43/101 [5:51:22<5:52:39, 364.82s/it][('Andre Myers', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Black-ish', 'Queridos blancos', 'Snowfall'], 'NULL'), ('Jerimiyah Dunbar', 'NULL', ['Reparto', 'Reparto'], ['su papel en S'], 'NULL'), ('Tom McLaren', 'NULL', ['Reparto', 'Dirección de casting', 'Producción'], ['su papel en Lost in Space', 'Expelled', 'El exorcismo de Molly Hartley'], ['Top Shorts Film Festival 2019 Ganador Honorable Mention Ensemble Blue Pig Compartido con: Yaxiong Shao · Eli Arenson · Tao Liu · Daniel Mentz · Leann Lei · Brent A. McCoy · Ana Corbi · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Calcutta International Cult Film Festival 2021 Ganador Outstanding Achievement Award Best Actor Into Temptation', 'DRUK International Film Festival 2021 Ganador DIFF Award Best Actor Into Temptation', 'Scorpiusfest 2020 Nominado Scorpius Award Best Casting - Short Film Blue Pig Compartido con: Brent A. McCoy · Ana Corbi · Mark Nelson · Rosanna Jean Foss · Miles Hayden · Helen Duffy · 2020 Nominado Scorpius Award Best Casting - Short Film Blue Pig Compartido con: Brent A. McCoy · Ana Corbi · Mark Nelson · Rosanna Jean Foss · Miles Hayden · Helen Duffy ·', 'Black Swan International Film Festival 2021 Ganador Outstanding Achievement Award Best Actor Into Temptation']), ('Rodney Chester', 'NULL', ['Reparto', 'Producción', 'Reparto'], ["su papel en Noah's Arc: Jumping the Broom", 'As I Am', "Noah's Arc"], 'NULL'), ('Piper Dellums', 'NULL', ['Producción', 'Guion', 'Reparto'], ['su papel en Strive', 'Cracka', 'Legacy of woman: Journey of the drum'], ['American Black Film Festival 2019 Ganador Best Screenplay Strive Compartido con: Sha-Risse Smith', 'Harlem International Film Festival 2019 Ganador Audience Award Strive Compartido con: Robert Rippberger · Tobias Deml · Ace Salvador · Hiroki Kamada · Scott M. Rosenfelt', "168 Film Festival, US 2019 Nominado Best Comedy Best Comedy Don't Talk About Jesus! Compartido con: Ian Eyre · Mary Ann Parker · Ulli Bonnekamp · Cici Couchois · Pat Couchois · Amelia Jackson-Gray · · · 2019 Ganador Audience Choice Award Audience Favorite Don't Talk About Jesus! Compartido con: Ian Eyre", 'Idyllwild International Festival of Cinema 2013 Ganador IIFC Award Best Documentary Short The Rent Party: Paying It Forward in Rhythm and Hues', 'Austin International Art Festival 2021 Ganador Festival Award Best women empowerment short Legacy of woman: Journey of the drum']), ('Rufus Tanksley', 'NULL', ['Reparto', 'Reparto'], ['su papel en As I Am', 'October', "October Director's Cut"], 'NULL'), ('Rob Hemen', 'NULL', ['Reparto', 'Reparto'], ['su papel en The System', 'Of Mummies and Martians', 'Kick Ass Cops: The Colgate Case'], 'NULL'), ('Tim Christian', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en The Virtual Connection', 'Tapu', 'Oasis'], ['Cinema Diverse, US 2021 Ganador Festival Favorite Short Film The Virtual Connection Compartido con: William Clift · Mel England · Antoine Perry · Adam W. Karwowski · Virginia Camarena · Christopher Null · ·', 'Independent Shorts Awards 2021 Nominado Honorable Mention Best Acting Ensemble Tapu Compartido con: Joshua Leomiti · Kyle Maddox · Jessica Staples · Katie Yandall', 'Hollywood Gold Awards 2021 Ganador Silver Award LGBTQ Film The Virtual Connection Compartido con: William Clift · Adam W. Karwowski · Mel England · Antoine Perry · Rachel Karp · Art Arutyunyan ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Mimi Frank', 'NULL', ['Reparto', 'Reparto'], ['su papel en As I Am', 'Adrift in Time', 'Murdered by Morning'], 'NULL'), ('Nicole Swirsky', 'NULL', ['Reparto', 'Reparto'], ['su papel en As I Am', 'The Holzer Files', 'Chapters of Our Lives'], 'NULL')]
 44%|████▎     | 44/101 [6:06:07<8:14:53, 520.93s/it][('Tom Cruise', '1962', ['Reparto', 'Producción', 'Dirección'], ['Top Gun: Ídolos del aire', 'El último samurái', 'Jerry Maguire'], ['Academy Awards, USA 2023 Nominado Oscar Best Motion Picture of the Year Top Gun: Maverick Compartido con: Christopher McQuarrie · David Ellison · Jerry Bruckheimer 2000 Nominado Oscar Best Actor in a Supporting Role Magnolia 1997 Nominado Oscar Best Actor in a Leading Role Jerry Maguire 1990 Nominado Oscar Best Actor in a Leading Role Nacido el cuatro de julio', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2022 Ganador Saturn Award Best Actor Top Gun: Maverick 2019 Nominado Saturn Award Best Actor Misión imposible: Fallout 2015 Nominado Saturn Award Best Actor Al filo del mañana 2012 Nominado Saturn Award Best Actor Misión: Imposible - Protocolo fantasma 2009 Nominado Saturn Award Best Actor Valkiria 7 más', 'Bambi Awards 2007 Ganador Bambi Courage Valkiria For using his passion and initiative to carry out ambitious and courageous film projects which would not have come to the screen without him.', 'Blockbuster Entertainment Awards 2001 Nominado Blockbuster Entertainment Award Favorite Actor - Action Misión: Imposible 2 2000 Nominado Blockbuster Entertainment Award Favorite Actor - Drama/Romance Eyes Wide Shut 2000 Ganador Blockbuster Entertainment Award Favorite Supporting Actor - Drama Magnolia 1997 Ganador Blockbuster Entertainment Award Favorite Actor - Comedy/Romance Jerry Maguire 1995 Ganador Blockbuster Entertainment Award Favorite Actor - Mystery/Thriller, On Video Entrevista con el vampiro', 'BAFTA Awards 1991 Nominado BAFTA Film Award Best Actor Nacido el cuatro de julio']), ('Jeremy Renner', '1971', ['Reparto', 'Producción', 'Sonido'], ['En tierra hostil', 'The Town: Ciudad de ladrones', 'Wind River'], ['Academy Awards, USA 2011 Nominado Oscar Best Performance by an Actor in a Supporting Role The Town: Ciudad de ladrones 2010 Nominado Oscar Best Performance by an Actor in a Leading Role En tierra hostil', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2019 Nominado Saturn Award Best Supporting Actor Vengadores: Endgame', 'Boston Society of Film Critics Awards 2009 Ganador BSFC Award Best Actor En tierra hostil', 'BAFTA Awards 2010 Nominado BAFTA Film Award Best Leading Actor En tierra hostil', 'Critics Choice Awards 2011 Nominado Critics Choice Award Best Supporting Actor The Town: Ciudad de ladrones 2010 Nominado Critics Choice Award Best Actor En tierra hostil']), ('Simon Pegg', '1970', ['Reparto', 'Guion', 'Producción'], ['Zombies Party (Una noche'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2016 Nominado Saturn Award Best Supporting Actor Misión: Imposible - Nación secreta 2014 Nominado Saturn Award Best Actor Bienvenidos al fin del mundo 2014 Nominado Saturn Award Best Writing Bienvenidos al fin del mundo Compartido con: Edgar Wright', 'American Comedy Awards, USA 2014 Nominado American Comedy Award Best Comedy Actor - Film Bienvenidos al fin del mundo 2014 Nominado American Comedy Award Best Comedy Screenplay Bienvenidos al fin del mundo Compartido con: Edgar Wright', "Boston Society of Film Critics Awards 2009 Ganador BSFC Award Best Ensemble Cast Star Trek Tied with Gabourey Sidibe, Mo'Nique, Paula Patton, 'Sherri Sheperd', Mariah Carey, Lenny Kravitz, Chyna Layne, , , and for . Compartido con: Chris Pine · Zachary Quinto · Zoe Saldana · Karl Urban · Leonard Nimoy · John Cho · Anton Yelchin · · · · · · · · ·", 'Bram Stoker Awards 2004 Ganador Bram Stoker Award Screenplay Zombies Party (Una noche... de muerte) Tied with Charlie Kaufman, Michel Gondry and Pierre Bismuth for ¡Olvídate de mí! (2004). Compartido con: Edgar Wright', 'BAFTA Awards 2002 Nominado BAFTA TV Award Situation Comedy Award Spaced For series II. Compartido con: Nira Park · Edgar Wright · Jessica Hynes']), ('Rebecca Ferguson', '1983', ['Reparto', 'Producción', 'Banda sonora'], ['El gran showman', 'Misión: Imposible - Nación secreta', 'Dune'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2024 Nominado Saturn Award Best Actress in a Television Series Silo 2021 Nominado Saturn Award Best Actress Doctor Sueño', 'Critics Choice Awards 2016 Nominado Critics Choice Award Best Actress in an Action Movie Misión: Imposible - Nación secreta', 'Golden Globes, USA 2014 Nominado Golden Globe Best Performance by an Actress in a Miniseries or Motion Picture Made for Television La reina blanca', 'Satellite Awards 2024 Nominado Satellite Award Best Actress in a Series, Drama or Genre Silo', 'Hamptons International Film Festival 2015 Ganador Breakthrough Performer Misión: Imposible - Nación secreta']), ('Ving Rhames', '1959', ['Reparto', 'Producción', 'Guion'], ['Pulp Fiction', 'Misión: Imposible 2', 'Al límite (Bringing Out the Dead'], ['Acapulco Black Film Festival 1998 Nominado Black Film Award Best Actor Rosewood', 'Blockbuster Entertainment Awards 2000 Nominado Blockbuster Entertainment Award Favorite Supporting Actor - Action La trampa', 'Primetime Emmy Awards 1998 Nominado Primetime Emmy Outstanding Lead Actor in a Miniseries or a Movie Don King: Only in America For playing: "Don King".', 'Golden Globes, USA 1998 Ganador Golden Globe Best Performance by an Actor in a Miniseries or Motion Picture Made for Television Don King: Only in America Ving Rhames asked Jack Lemmon, who was also nominated, on the stage and tributed the award to him.', 'Satellite Awards 2000 Nominado Golden Satellite Award Best Performance by an Actor in a Supporting Role, Comedy or Musical Al límite (Bringing Out the Dead) 1998 Nominado Golden Satellite Award Best Actor in a Miniseries or a Motion Picture Made for Television Don King: Only in America']), ('Sean Harris', '1966', ['Reparto', 'Guion', 'Dirección'], ['Misión: Imposible - Nación secreta', 'Harry Brown', 'Misión imposible: Fallout'], ['Film Critics Circle of Australia Awards 2023 Ganador FCCA Award Best Supporting Actor El extraño', 'BAFTA Awards 2014 Ganador BAFTA TV Award Best Leading Actor Southcliffe', "British Independent Film Awards 2016 Nominado British Independent Film Award Best Supporting Actor Código criminal 2015 Nominado British Independent Film Award Best Supporting Actor Macbeth 2014 Nominado British Independent Film Award Best Supporting Actor '71", 'Chlotrudis Awards 2003 Nominado Chlotrudis Award Best Supporting Actor 24 Hour Party People', 'Austin Fantastic Fest 2011 Ganador Horror Jury Prize Best Actor - Feature Un lugar solitario para morir']), ('Simon McBurney', '1957', ['Reparto', 'Equipos adicionales', 'Guion'], ['El topo', 'El último rey de Escocia', 'El mensajero del miedo'], 'NULL'), ('Jingchu Zhang', '1980', ['Reparto', 'Dirección', 'Guion'], ['Misión: Imposible - Nación secreta', 'Protégé', 'Hora punta 3'], ['Cairo International Film Festival 2006 Ganador Best Actress Fang xiang zhi lu', 'Golden Bauhinia Awards 2006 Nominado Golden Bauhinia Best Supporting Actress Siete espadas', 'Golden Horse Film Festival 2010 Nominado Golden Horse Award Best Supporting Actress Tang shan da di zhen', 'Golden Rooster Awards 2013 Nominado Golden Rooster Best Actress Wo de ying zi zai ben pao 2011 Nominado Golden Rooster Best Supporting Actress Tang shan da di zhen 2005 Nominado Golden Rooster Best Actress Hua yao xin niang', 'Hong Kong Film Awards 2019 Nominado Hong Kong Film Award Best Actress Mou seung 2011 Nominado Hong Kong Film Award Best Supporting Actress Chun sing gai bei 2010 Nominado Hong Kong Film Award Best Actress Tin shui wai dik ye yu mo 2008 Nominado Hong Kong Film Award Best Actress Protégé 2006 Nominado Hong Kong Film Award Best Supporting Actress Siete espadas']), ('Tom Hollander', '1967', ['Reparto', 'Producción', 'Guion'], ['Pride & Prejudice (Orgullo y prejuicio', 'Gosford Park', 'In the loop'], ['Banff Television Festival 2011 Nominado Banff Rockie Award Best Sitcom Rev. UK Compartido con: James Wood · Peter Cattaneo · Kenton Allen', 'Biarritz International Festival of Audiovisual Programming 2004 Ganador Golden FIPA TV Series and Serials: Actor Espías de Cambridge', 'BAFTA Awards 2017 Ganador BAFTA TV Award Best Supporting Actor El infiltrado 2015 Nominado BAFTA TV Award Best Male Performance in a Comedy Programme Rev. 2012 Nominado BAFTA TV Award Best Male Performance in a Comedy Programme Rev. 2012 Nominado BAFTA TV Award Best Situation Comedy Rev. Compartido con: James Wood · Peter Cattaneo 2011 Nominado BAFTA TV Award Best Male Performance in a Comedy Role Rev. 1 más', 'British Comedy Awards 2012 Nominado British Comedy Award Best TV Comedy Actor Rev. 2010 Nominado British Comedy Award Best TV Comedy Actor Rev.', 'British Independent Film Awards 2009 Nominado British Independent Film Award Best Supporting Actor In the loop 2005 Nominado British Independent Film Award Best Supporting Actor/Actress The libertine']), ('Jens Hultén', '1963', ['Reparto', 'Reparto'], ['Misión: Imposible - Nación secreta', 'Skyfall', 'Alpha'], 'NULL')]
 45%|████▍     | 45/101 [6:09:09<6:31:12, 419.16s/it][('Joe Brennan', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Eye'], 'NULL'), ('Robert Factor', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Mourning in China', 'Pleasures', 'Stuck'], ['Bare Bones International Film & Music Festival 2014 Nominado Bonehead Award Best Family Dramedy Micro Short Mourning in China Compartido con: Tony Chiu 2011 Nominado Bonehead Award Actor/Actress Category---Best Actor/Actress in a Movie Scene Stuck', 'Accolade Competition 2013 Ganador Award of Merit Short Film Mourning in China Compartido con: Tony Chiu 2010 Ganador Award of Merit Lead Actor Pleasures 2010 Ganador Award of Merit Short Film Pleasures Compartido con: Lillian Kho · Ellen Ho 2007 Ganador Honorable Mention Short Film Stuck Compartido con: David Warshawski · Joseph Candido · Kevin Candido · Charmel Green · Bill Weisman · Brent Roske 2007 Ganador Honorable Mention Original Screenplay Stuck', 'The Indie Gathering International Film Festival 2013 Nominado Honorable Mention Short Film-Drama Mourning in China Compartido con: Tony Chiu 2012 Ganador 1st Place Indie Gathering Award Best Microfilm Drama Lost in China 2012 Nominado 1st Place Indie Gathering Award Best Short Film - Drama Pill Bottle Angels Compartido con: Karen Bullis · Carrie Lynn Certa · Daniel Noh · Joshua Noh 2012 Nominado 1st Place Indie Gathering Award Short Scripts - Drama Mourning in China 2011 Nominado 1st Place Indie Gathering Award Best Short Script - Drama Lost in China 3 más', "Action on Film International Film Festival, USA 2016 Ganador Action on Film Award Best Supporting Actor - Short Permission 2016 Ganador Action on Film Award Stella Adler Award for Excellence in Film Career Body of Work 2016 Nominado Action on Film Award Best Supporting Actor - Short Permission 2013 Nominado Action on Film Award Best Produced Screenplay - Short Mourning in China Compartido con: Tony Chiu 2012 Nominado Action on Film Award Best Villain The Chronicles of the Order: Xander's Path 11 más", "Nevada International Film Festival, US 2013 Ganador Platinum Reel Award Actor's Showcase Competition Mourning in China"])]
 46%|████▌     | 46/101 [6:22:09<8:03:29, 527.45s/it][('Ilenia Pastorelli', '1985', ['Reparto', 'Reparto'], ['Le llamaban Jeeg Robot', 'Bendita locura', 'Dark Glasses'], 'NULL'), ('Matilde Gioli', '1989', ['Reparto', 'Reparto'], ['El capital humano', "Gli uomini d'oro", 'Runner'], ["Italian National Syndicate of Film Journalists 2020 Nominado Silver Ribbon Best Supporting Actress (Migliore Attrice Non Protagonista) Gli uomini d'oro 2014 Ganador Guglielmo Biraghi Award El capital humano", 'CinEuphoria Awards 2024 Nominado CinEuphoria Best Actress in a Short Film - International Competition Frammenti', 'BIFEST - Bari International Film Festival 2014 Ganador Italian Competition Award Best Supporting Actress El capital humano', 'Golden Ciak Awards 2014 Nominado Golden Ciak Best Supporting Actress (Migliore Attrice Non Protagonista) El capital humano', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Giuseppe Maggio', '1992', ['Reparto', 'Banda sonora', 'Reparto'], ['Explota, explota', 'En el mejor momento', 'Un amore così grande'], 'NULL'), ('Matteo Martari', '1983', ['Reparto', 'Reparto'], ['Los Médici: Señores de Florencia', 'A un paso del cielo', 'Mal genio'], 'NULL'), ('Marta Gastini', '1989', ['Reparto', 'Producción', 'Guion'], ['El rito', 'Drácula 3D', 'Borgia'], ['Giffoni Film Festival 2014 Ganador Giffoni Award Explosive Talent', 'Italian National Syndicate of Film Journalists 2011 Nominado Silver Ribbon Best Supporting Actress (Migliore Attrice Non Protagonista) El rito', "Venice Film Festival 2011 Nominado L'Oreal Paris Award El rito", 'California Film Awards 2013 Ganador Grand Winner Best Actress Tanith', "Golden Ciak Awards 2021 Nominado Golden Ciak Best Performance in a Tv-series (Protagonista dell'anno in una serie Tv) Tutta colpa di Freud"]), ('Flavio Furno', '1986', ['Reparto', 'Reparto'], ['Il candidato', 'Mujer y marido', 'Per amore del mio popolo'], 'NULL'), ('Nicola La Gioia', 'NULL', ['Reparto', 'Reparto'], ['su papel en 4 mitades', 'Indro'], 'NULL'), ('Armando Quaranta', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cero', '4 mitades', 'Te presento a Sofía'], 'NULL'), ('Marta Malvestiti', 'NULL', ['Reparto', 'Reparto'], ['su papel en 4 mitades'], 'NULL'), ('Tommaso Basili', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ferrari', 'Ottoman Rising', 'Lamborghini: El hombre detrás de la leyenda'], 'NULL')]
 47%|████▋     | 47/101 [6:23:48<5:59:07, 399.04s/it][('Donnie Pierre', '1982', ['Reparto', 'Dirección', 'Producción'], ['Stuck in the Cycle', 'Philosophy of Hate', "Bryan Matthew Ward's Voyeur"], 'NULL')]
 48%|████▊     | 48/101 [6:36:42<7:31:45, 511.42s/it][('Ashraf Barhom', '1979', ['Reparto', 'Equipos adicionales', 'Reparto'], ['La sombra del reino', 'Coriolanus', 'Furia de titanes'], 'NULL'), ('Jwan Alqatami', 'NULL', ['Reparto', 'Reparto'], ['su papel en Exodus'], 'NULL'), ('Isa Aouifia', '1974', ['Reparto', 'Guion', 'Banda sonora'], ['Taelgia', 'Exodus', 'Heder'], 'NULL'), ('Leis Yasin', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ring mamma!', 'The Detective from Beledweyne (Serie de TV', 'Exodus'], 'NULL'), ('Khaled Alqatami', 'NULL', ['Reparto', 'Reparto'], ['su papel en Exodus'], 'NULL'), ('Olle Wirenhed', '1971', ['Producción', 'Reparto', 'Dirección de la segunda unidad o asistencia a dirección'], ['Fikon', 'Pojkarna', 'Index'], ['Antalya Golden Orange Film Festival 2015 Ganador International Competition Best Screenplay Award Pojkarna Compartido con: Alexandra-Therese Keining · Helena Wirenhed', 'Drama Short Film Festival 2019 Ganador Golden Dionysus Award Best Fiction Film (National) Index Compartido con: Nicolas Kolovos · Helena Wirenhed 2015 Ganador Best Fiction Film (National) Best Film Fikon Compartido con: Nicolas Kolovos', 'Hellenic Film Academy Awards 2020 Nominado Hellenic Film Academy Award Best Film To thavma tis thalassas ton Sargasson Compartido con: Maria Drandaki · Titus Kreyenberg · Ellen Havenith · Anthony Muir 2016 Ganador Hellenic Film Academy Award Best Short Film Fikon Compartido con: Nicolas Kolovos', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Antonis Tsiotsiopoulos', 'NULL', ['Reparto', 'Guion', 'Equipos adicionales'], ['su papel en Magnitika pedia', 'W', '5pm Seaside'], ['Drama Short Film Festival 2022 Ganador Best Actor 5pm Seaside', 'Athens International Film Festival 2022 Ganador Best Male Performance Award Best Actor 5pm Seaside', 'Hellenic Film Academy Awards 2022 Nominado Hellenic Film Academy Award Best Actor Magnitika pedia 2022 Ganador Hellenic Film Academy Award Best Screenplay Magnitika pedia Compartido con: Yorgos Goussis · Elena Topalidou', 'Grand Off World Independent Film Awards 2016 Nominado Grand OFF Award Best Actor A Buddy Already Dead', 'Mostra de Curtas Noia 2020 Ganador Jury Award Best Actor W 2020 Ganador Jury Award Best Actor W']), ('Nasim Alatras', 'NULL', ['Dirección', 'Reparto', 'Guion'], ['su papel en O askos tou Aiolou', 'Exodus'], 'NULL'), ('Andreas Zikoulis', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sta syrmata', 'Abejas Salvajes', 'Eteros ego'], 'NULL'), ('Valentina Hilani', 'NULL', ['Reparto', 'Reparto'], ['su papel en Exodus'], 'NULL')]
 49%|████▊     | 49/101 [6:50:02<8:38:20, 598.09s/it][('Tak Sakaguchi', '1975', ['Reparto', 'Equipos adicionales', 'Especialistas'], ['Versus', 'Sakigake!! Otokojuku', 'Gokudô heiki'], 'NULL'), ('Asami', '1985', ['Reparto', 'Efectos especiales', 'Banda sonora'], 'NULL', 'NULL'), ('Naoto Takenaka', '1956', ['Reparto', 'Música', 'Dirección'], ['Munô no hito', '119', 'Shall We Dance?'], ['Hochi Film Awards 1991 Ganador Hochi Film Award Best New Director Munô no hito', 'Awards of the Japanese Academy 2001 Nominado Award of the Japanese Academy Best Actor Actor secundario 1998 Nominado Award of the Japanese Academy Best Actor Tokyo biyori 1998 Nominado Award of the Japanese Academy Best Director Tokyo biyori 1997 Ganador Award of the Japanese Academy Best Supporting Actor Shall We Dance? 1996 Ganador Award of the Japanese Academy Best Supporting Actor East Meets West 5 más', 'Nantes Three Continents Festival 1997 Nominado Golden Montgolfiere Tokyo biyori 1991 Ganador Special Jury Award Munô no hito Tied with Mama (1990). 1991 Ganador Award of the City of Nantes Munô no hito Tied with Mama (1990). 1991 Nominado Golden Montgolfiere Munô no hito', 'Venice Film Festival 1991 Ganador FIPRESCI Prize Parallel Sections Munô no hito', "Kinema Junpo Awards 1996 Ganador Kinema Junpo Award Best Supporting Actor East Meets West Tied with Mickey Curtis for Kamikaze takushî (1995). 1995 Ganador Readers' Choice Award Best Film 119"]), ('Yoshihiro Nishimura', '1967', ['Maquillaje', 'Efectos especiales', 'Producción'], ['Tôkyô zankoku keisatsu', 'Kodoku: Mîtobôru mashin', 'The ABCs of Death'], ['Chicago International Film Festival 2012 Nominado Gold Hugo After Dark Competition The ABCs of Death Compartido con: Kaare Andrews · Angela Bettis · Adrian Garcia Bogliano · Bruno Forzani · Hélène Cattet · Ernesto Díaz Espinoza · · · · · · · · · · · · · · · · · · · ·', 'Fantasia Film Festival 2009 Nominado Guru Prize for Most Energetic Film Vampire Girl vs. Frankenstein Girl Tied with Fireball (2009). Compartido con: Naoyuki Tomomatsu 2008 Ganador Best Asian Film Best Film Tôkyô zankoku keisatsu', 'Puchon International Fantastic Film Festival 2008 Ganador European Fantastic Film Festival Federation Asian Award - Special Mention Tôkyô zankoku keisatsu', 'Molins Film Festival 2017 Nominado Audience Award Being Different Kodoku: Mîtobôru mashin', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Noboru Iguchi', '1969', ['Dirección', 'Guion', 'Reparto'], ['The ABCs of Death', 'Kataude mashin gâru', 'Sentô shôjo: Chi no tekkamen densetsu'], 'NULL'), ('Yumi Sugimoto', '1989', ['Reparto', 'Banda sonora', 'Reparto'], ['Enjin sentai Gôonjâ', 'Samurai sentai Shinkenjâ tai Gôonjâ Ginmakuban!', 'Kaizoku sentai Gôkaijâ'], 'NULL'), ('Hikaru Shida', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sentô shôjo: Chi no tekkamen densetsu', 'Robo-geisha', 'Oz Academy'], 'NULL'), ('Kanji Tsuda', '1965', ['Reparto', 'Dirección', 'Guion'], ['La maldición (The grudge', 'Zatoichi', 'Hana-bi'], 'NULL'), ('Cay Izumi', '1986', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Tôkyô zankoku keisatsu', 'Sentô shôjo: Chi no tekkamen densetsu', 'Robo-geisha'], 'NULL'), ('Suzuka Morita', '1992', ['Reparto', 'Reparto'], ['Kamen Raidâ × Sûpâ Sentai × Uchû Keiji: Supâ Hîrô Taisen Z', 'Samurai Sentai Shinkenjâ', 'Uchû Keiji Gyaban vs'], 'NULL')]
 50%|████▉     | 50/101 [7:02:58<9:13:40, 651.38s/it][('Jirí Langmajer', '1966', ['Reparto', 'Banda sonora', 'Reparto'], ['El idiota', 'Ordinace v ruzové zahrade', 'Pravidla lzi'], 'NULL'), ('Ivana Chýlková', '1963', ['Reparto', 'Banda sonora', 'Reparto'], ['Díky za kazdé nové ráno', 'Cas sluhu', 'Prítelkyne z domu smutku'], 'NULL'), ('Lucie Zácková', '1978', ['Reparto', 'Reparto'], ['Kobry a uzovky', 'Karamazovi', 'Rapl'], 'NULL'), ('Iva Janzurová', '1941', ['Reparto', 'Banda sonora', 'Reparto'], ['Ene bene', 'Výlet', 'Co chytnes v zite'], ["Art Film Festival 1999 Ganador Actor's Mission Award", 'Czech Lions 2020 Nominado Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Teroristka 2003 Ganador Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Výlet 2001 Nominado Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Ene bene 1999 Ganador Czech Lion Best Actress (Zenský herecký výkon v hlavní roli) Co chytnes v zite', "Karlovy Vary International Film Festival 2015 Ganador Festival President's Award For outstanding contribution to Czech cinema.", 'Pilsen Film Festival 1999 Ganador Audience Award Best Actress Co chytnes v zite', 'Sochi International Film Festival 2000 Ganador Best Actress Ene bene Tied with Tinna Gunnlaugsdóttir for Ungfrúin góða og húsið (1999).']), ('Pavel Nový', '1948', ['Reparto', 'Reparto'], ['Amadeus', 'Bába z ledu', 'Amerika'], 'NULL'), ('Martin Pechlát', '1974', ['Reparto', 'Banda sonora', 'Reparto'], ['Odpad mesto smrt', 'Okupace', 'Atlas ptáku'], 'NULL'), ('Andrej Polák', '1979', ['Reparto', 'Reparto'], ['Operación Anthropoid', 'Whiskey Cavalier', 'El pájaro pintado'], 'NULL'), ('Karolína Lipowská', '2005', ['Reparto', 'Reparto'], ['The Show', 'Chlap na strídacku', 'Bajkeri'], 'NULL'), ('Oliver Vyskocil', 'NULL', ['Reparto', 'Reparto'], ['su papel en Chlap na strídacku', 'Jedna rodina', 'Bábovky'], 'NULL'), ('Tereza Taliánová', '2005', ['Reparto', 'Reparto'], ['Chlap na strídacku', 'Spindl', 'Spindl 2'], 'NULL')]
 50%|█████     | 51/101 [7:15:32<9:28:21, 682.03s/it][('Ace Malik', 'NULL', ['Reparto', 'Reparto'], ['su papel en Nightfall: Second World III', 'Second World: Blood, Fire and Smoke'], 'NULL'), ('Altorro Prince Black', 'NULL', ['Reparto', 'Dirección', 'Casting'], ['su papel en Boundary', 'House of Cards', 'City of Lost Souls'], ["LA Web Fest 2011 Ganador Outstanding Achievement Award Outstanding Lead Actor In A Mockumentary Series Sista Wives: Jacob's House", "World Music and Independent Film Festival 2015 Nominado Festival Award Best Actor - Feature Film Bachelorette's Degree 2015 Nominado Festival Award Best Director - Feature Film Bachelorette's Degree 2013 Ganador Festival Award Best Supporting Actor in a Short Film / DMV Table for Three 2013 Nominado Festival Award Best Supporting Actor in a Feature / DMV Dead Reckoning", 'Television, Internet and Video Association of DC (TIVA-DC) Peer Awards 2016 Ganador Peer Award Acting on Camera - Fiction, Male City of Lost Souls 2016 Ganador Peer Award Acting on Camera - Fiction, Male Reunion', 'Hot Media International Film Festival, US 2012 Ganador Festival Award Best Supporting Actor', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Amanda Zielsdorf', 'NULL', ['Reparto', 'Reparto'], ['su papel en Second World: Blood, Fire and Smoke'], 'NULL'), ('Aries Zeng', 'NULL', ['Reparto', 'Reparto'], ['su papel en Second World: Blood, Fire and Smoke'], 'NULL'), ('Bailey R. Center', 'NULL', ['Reparto', 'Reparto'], ['su papel en E-Preeti', 'Nightfall: Second World III', 'Clowns'], 'NULL'), ('Brandon Dorsey', 'NULL', ['Reparto', 'Vestuario', 'Producción'], ['su papel en Cat Fight', 'Nightfall: Second World III', 'Second World: Blood, Fire and Smoke'], 'NULL'), ('Brandon Michaux', 'NULL', ['Reparto', 'Reparto'], ['su papel en Haze', 'Sons of the City', 'Second World: Blood, Fire and Smoke'], 'NULL'), ('Briyan Moreno', 'NULL', ['Reparto', 'Reparto'], ["su papel en Bachelorette's Degree", 'Dead Reckoning', 'Second World: Blood, Fire and Smoke'], 'NULL'), ('Carl Allen', 'NULL', ['Reparto', 'Reparto'], ['su papel en 6 Nonsmokers', 'Second World: Blood, Fire and Smoke'], 'NULL'), ('Carl L. Allen', 'NULL', ['Reparto', 'Reparto'], ['su papel en Second World: Blood, Fire and Smoke'], 'NULL')]
 51%|█████▏    | 52/101 [7:22:09<8:07:18, 596.70s/it][('Bianca Boyd', 'NULL', ['Reparto', 'Reparto'], ['su papel en Standing Up for Sunny', 'Rip Tide', 'The Secret Letters Project'], 'NULL'), ('Sophie Lowe', 'NULL', ['Reparto', 'Edición', 'Producción'], ['Derribad al hombre', 'Medieval', 'The Dive'], ['Film Critics Circle of Australia Awards 2010 Nominado FCCA Award Best Supporting Actor - Female Beautiful Kate', 'Australian Film Institute 2009 Nominado AFI Award Best Lead Actress Beautiful Kate', 'Screen Music Awards, Australia 2010 Nominado Screen Music Award Best Original Song Composed for a Feature Film, Telemovie, TV Series or Mini-Series Road Train Compartido con: Rafael May', 'The Equity Ensemble Awards 2012 Ganador The Equity Award Outstanding Performance by an Ensemble in a Mini-series or Telemovie The Slap Compartido con: Raffaele Costabile · Blake Davis · Essie Davis · Alex Dimitriades · Melissa George · Diana Glenn · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Nicola Panopoulos', 'NULL', ['Reparto', 'Reparto'], ['su papel en Deluge'], 'NULL'), ('Tysan Towney', '1989', ['Reparto', 'Reparto'], ['The Leftovers', 'Cleverman', 'Carpark Clubbing'], 'NULL'), ('Skye Wansey', 'NULL', ['Reparto', 'Reparto'], ['su papel en Chopper', 'Farscape', 'Top of the Lake'], 'NULL')]


NameError: name 'dic_peli' is not defined